In [1]:

import io
import os
import pandas as pd
import ast
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle, Arc
from IPython.display import display
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
import json
from collections import defaultdict
import mezzala
import itertools
import json
import urllib.request
from datetime import datetime  
from collections import defaultdict
from collections import Counter
from scipy import stats
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from PIL import Image
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd
import unidecode
import re



In [2]:
# Definir la ruta relativa desde la ubicación actual (notebooks) hasta la carpeta de datos
data_path = "../../data_backup/archivos_parquet"

# Obtener la lista de archivos parquet que contienen "all"
parquet_files = [f for f in os.listdir(data_path) if f.endswith('.parquet') and 'all' in f.lower()]

# Procesar cada archivo y crear variables globales
for file in parquet_files:
    # Crear el nombre del DataFrame (reemplazar .parquet y agregar prefijo df_)
    df_name = 'df_' + file.replace('.parquet', '')
    
    # Leer el archivo parquet
    file_path = os.path.join(data_path, file)
    
    # Crear la variable global usando globals()
    globals()[df_name] = pd.read_parquet(file_path)
    
    # Imprimir información sobre el proceso
    print(f"Archivo procesado: {file}")
    print(f"Nombre del DataFrame: {df_name}")
    print(f"Dimensiones: {globals()[df_name].shape}")
    print("-" * 50)

Archivo procesado: seasons_teams_all.parquet
Nombre del DataFrame: df_seasons_teams_all
Dimensiones: (105, 7)
--------------------------------------------------
Archivo procesado: formations_league_all.parquet
Nombre del DataFrame: df_formations_league_all
Dimensiones: (40707, 9)
--------------------------------------------------
Archivo procesado: names_players_league_all.parquet
Nombre del DataFrame: df_names_players_league_all
Dimensiones: (9561, 12)
--------------------------------------------------
Archivo procesado: leagues_all.parquet
Nombre del DataFrame: df_leagues_all
Dimensiones: (42, 7)
--------------------------------------------------
Archivo procesado: sequence_data_league_all.parquet
Nombre del DataFrame: df_sequence_data_league_all
Dimensiones: (244675, 7)
--------------------------------------------------
Archivo procesado: players_league_all.parquet
Nombre del DataFrame: df_players_league_all
Dimensiones: (9584, 6)
--------------------------------------------------
A

In [3]:
print(df_player_stats_extended_league_all.columns)

Index(['goal', 'total_shot', 'shot_on_target', 'shot_off_target',
       'shot_blocked', 'shot_in_PA', 'shot_outside_of_PA', 'free_kick',
       'corner_kick', 'throw_in', 'penalty_kick', 'pass', 'pass_succeeded',
       'pass_failed', 'assist', 'key_pass', 'control_under_pressure', 'cross',
       'cross_succeeded', 'final_third_area_pass',
       'final_third_area_pass_succeeded', 'middle_area_pass',
       'middle_area_pass_succeeded', 'defensive_area_pass',
       'defensive_area_pass_succeeded', 'short_pass', 'short_pass_succeeded',
       'medium_range_pass', 'medium_range_pass_succeeded', 'long_pass',
       'long_pass_succeeded', 'forward_pass', 'forward_pass_succeeded',
       'backward_pass', 'backward_pass_succeeded', 'sideways_pass',
       'sideways_pass_succeeded', 'tackle', 'tackle_succeeded', 'aerial_duel',
       'aerial_duel_succeeded', 'aerial_duel_failed', 'ground_duel',
       'ground_duel_succeeded', 'ground_duel_failed', 'loose_ball_duel',
       'loose_ball_duel

In [4]:
print(df_names_players_league_all.columns)

Index(['player_id', 'root_player_id', 'player_name', 'player_last_name',
       'player_name_en', 'player_last_name_en', 'back_number', 'player_role',
       'main_position', 'team_id', 'birth_date', 'season_id'],
      dtype='object')


In [5]:

def find_league_name(row, df_league_all):
    season_id = str(row['season_id'])
    
    for _, league_row in df_league_all.iterrows():
        # Maneja tanto strings de listas como de arrays numpy
        season_ids_str = league_row['season_ids']
        try:
            # Si es una representación string de una lista normal
            season_ids = ast.literal_eval(season_ids_str)
        except ValueError:
            # Si es una representación string de array numpy
            if 'array(' in season_ids_str:
                # Extrae números del string del array
                numbers = season_ids_str.replace('array([', '').replace('])', '').split(',')
                season_ids = [int(num.strip()) for num in numbers]
            else:
                continue
        
        if season_id in map(str, season_ids):
            return league_row['name']
    
    return None


# Paso 1: Realizar el merge inicial
df_combined = pd.merge(
    df_player_stats_extended_league_all,
    df_names_players_league_all[['player_id', 'player_name_en', 'player_last_name_en', 'main_position', 'back_number']],
    on='player_id',
    how='left'
)

# Paso 1B: Función para formatear nombres (primera letra mayúscula, resto minúscula)
def format_name(name):
    if pd.isna(name) or name == '0':
        return name
    return ' '.join(word.capitalize() for word in name.split())

# Paso 1C: Aplicar formato a los nombres
df_combined['player_name_en'] = df_combined['player_name_en'].apply(format_name)
df_combined['player_last_name_en'] = df_combined['player_last_name_en'].apply(format_name)

# Paso 1D: Función para buscar nombres en df_events_league_all
def find_name(row, events_df):
    if pd.isna(row['player_name_en']) or row['player_name_en'] == '0' or pd.isna(row['player_last_name_en']) or row['player_last_name_en'] == '0':
        player_id = row.name  # Asumiendo que el índice es el player_id
        match = events_df[events_df['player_id'] == player_id]
        if not match.empty:
            return pd.Series({
                'player_name_en': format_name(match['player_name'].iloc[0]) if 'player_name' in match.columns else row['player_name_en'],
                'player_last_name_en': format_name(match['player_last_name'].iloc[0]) if 'player_last_name' in match.columns else row['player_last_name_en']
            })
    return row[['player_name_en', 'player_last_name_en']]

# Paso 1E: Aplicar la función de búsqueda
df_combined[['player_name_en', 'player_last_name_en']] = df_combined.apply(lambda row: find_name(row, df_events_league_all), axis=1)

# Paso 2: Merge el resultado con df_teams_league_all
df_combined = pd.merge(
    df_combined,
    df_teams_league_all[['team_id', 'name']],
    on='team_id',
    how='left'
)

# Renombrar la columna 'name' a 'team_name' para mayor claridad
df_combined = df_combined.rename(columns={'name': 'team_name'})

# Paso 3: Aplicar la función para encontrar el nombre de la liga
df_combined['league_name'] = df_combined.apply(lambda row: find_league_name(row, df_leagues_all), axis=1)

# Paso 4: Merge con df_matches_all para añadir información adicional
df_combined = pd.merge(
    df_combined,
    df_matches_league_all[['match_id', 'home_team_id', 'away_team_id', 'home_team_name', 'away_team_name', 'round_name', 'season_name']],
    on='match_id',
    how='left'
)

# Paso 5: Crear columnas para el nombre del equipo y condición de local/visitante
df_combined['equipo_partido'] = np.where(
    df_combined['team_id'] == df_combined['home_team_id'],
    df_combined['home_team_name'],
    df_combined['away_team_name']
)

df_combined['home_away'] = np.where(
    df_combined['team_id'] == df_combined['home_team_id'],
    'local',
    'visitante'
)

# Nuevo Paso 6: Merge con df_lineups_league_all y crear la columna starter_sub
df_combined = pd.merge(
    df_combined,
    df_lineups_league_all[['match_id', 'season_id', 'player_id', 'is_starting_lineup']],
    on=['match_id', 'season_id', 'player_id'],
    how='left'
)

df_combined['starter_sub'] = np.where(
    df_combined['is_starting_lineup'] == True,
    'starter',
    'sub'
)

# Reemplazar valores nulos por 0 (excepto en starter_sub)
for column in df_combined.columns:
    if column != 'starter_sub':
        df_combined[column] = df_combined[column].fillna(0)

# Eliminar filas duplicadas
df_combined = df_combined.drop_duplicates()

# Verificar las columnas del DataFrame final
df_combined

KeyboardInterrupt: 

In [ ]:
# Obtener los valores únicos de la columna 'main_position'
df_combined['season_name'].unique()

In [ ]:

# Lista de columnas a mantener
columnas_a_mantener = [
    'goal', 'match_id','total_shot', 'shot_on_target', 'shot_in_PA', 'shot_outside_of_PA',
    'free_kick', 'penalty_kick', 'corner_kick', 'pass', 'pass_succeeded',
    'assist', 'key_pass', 'control_under_pressure', 'cross',
    'cross_succeeded', 'final_third_area_pass', 'final_third_area_pass_succeeded',
    'middle_area_pass', 'middle_area_pass_succeeded', 'defensive_area_pass',
    'defensive_area_pass_succeeded', 'long_pass', 'long_pass_succeeded',
    'forward_pass', 'forward_pass_succeeded', 'backward_pass',
    'backward_pass_succeeded', 'sideways_pass', 'sideways_pass_succeeded',
    'tackle', 'tackle_succeeded', 'aerial_duel', 'aerial_duel_succeeded',
    'ground_duel', 'ground_duel_succeeded', 'loose_ball_duel',
    'loose_ball_duel_succeeded', 'block', 'intercept', 'clearance', 'recovery',
    'intervention', 'take_on', 'take_on_succeeded', 'mistake',
    'foul', 'offside', 'yellow_card', 'red_card', 'own_goal',
    'goal_conceded', 'aerial_clearance', 'aerial_clearance_succeeded',
    'defensive_line_support', 'defensive_line_support_succeeded',
    'rating', 'save_by_catching', 'save_by_punching','play_time'
]

# Cambiar los nombres de las columnas del nombre del jugador
df_combined = df_combined.rename(columns={
    'player_name_en': 'player_name',
    'player_last_name_en': 'player_last_name'
})
# Crear la nueva columna 'match'
df_combined['match'] = df_combined['home_team_name'] + '-' + df_combined['away_team_name']

# Añadir las columnas que agregamos en los pasos anteriores
columnas_adicionales = ['player_id', 'player_name', 'player_last_name', 'back_number', 'main_position', 'team_name', 'league_name', 'season_name', 'round_name', 'match','home_away', 'starter_sub']

# Combinar todas las columnas que queremos mantener
todas_columnas = columnas_a_mantener + columnas_adicionales

# Seleccionar solo estas columnas de df_combined y filtrar por defensas centrales
df_players = df_combined[todas_columnas]

# Convertir play_time de milésimas de segundos a minutos enteros
df_players['play_time_minutes'] = np.round(df_players['play_time'] / (1000 * 60)).astype(int)

# Contar el número de valores nulos por columna antes de reemplazarlos
null_counts_before = df_combined.isnull().sum()
print("Número de valores nulos por columna antes de reemplazarlos:")
print(null_counts_before)
print("\n")

# Reemplazar valores nulos por 0 (excepto en back_number)
for column in df_combined.columns:
    if column != 'back_number':
        df_combined[column] = df_combined[column].fillna(0)
        

# Mostrar información sobre el DataFrame
df_players

In [ ]:

df_players['player'] = df_players['player_name'].astype(str) + ' ' + df_players['player_last_name'].astype(str)
df_players.fillna(0, inplace=True)
df_players  =df_players[(df_players['main_position'] != 0) & (df_players['player'] != '0 0')]

df_players 

In [ ]:
df_players ['season_name'].unique()

In [ ]:


def create_kpi_players(df_players):
    print("Iniciando la creación de df_KPI_players...")
    print("Columnas disponibles en df_players:")
    print(df_players.columns)
    
    required_columns = ['match_id', 'match','team_name', 'league_name', 'season_name', 'round_name',
                        'home_away','player_id', 'player', 'main_position', 'back_number', 'starter_sub', 'play_time_minutes']
    
    missing_columns = [col for col in required_columns if col not in df_players.columns]
    if missing_columns:
        print("Columnas faltantes:")
        print(missing_columns)
        raise ValueError(f"Las siguientes columnas no están presentes en df_players: {missing_columns}")
    
    df_KPI_players = df_players[required_columns].copy()
    
    df_KPI_players['participacion'] = df_KPI_players.apply(lambda row:
        (10 if row['play_time_minutes'] >= 90 else
         0 if row['play_time_minutes'] == 0 else
         (row['play_time_minutes'] / 90) * 10) +
        (3 if row['starter_sub'] == 'starter' else 0),
        axis=1
    ).round(2)
    
    def safe_divide(a, b):
        return np.round(np.where(b != 0, a / b, 0), 2)
    
    df_KPI_players['%_tiros_dentro_area'] = safe_divide(df_players['shot_in_PA'], df_players['total_shot'])
    df_KPI_players['%_tiros_fuera_area'] = safe_divide(df_players['shot_outside_of_PA'], df_players['total_shot'])
    df_KPI_players['Capacidad_disparos'] = df_KPI_players['%_tiros_fuera_area'] +  df_KPI_players['%_tiros_dentro_area']
    
    df_KPI_players['lanzador_ABP'] = (df_players['free_kick'] +  df_players['corner_kick'] + df_players['penalty_kick']).round(2)
    df_KPI_players['precision_pase'] = safe_divide(df_players['pass_succeeded'], df_players['pass'])
    df_KPI_players['influencia_pases_gol'] = (df_players['assist'] + df_players['key_pass']+ df_players['cross_succeeded']).round(2)
    
    df_KPI_players['controles_bajo_presion'] = df_players['control_under_pressure'].round(2)

    
    df_KPI_players['Eficacia_disparos'] = safe_divide(df_players['shot_on_target'], df_players['total_shot'])
    df_KPI_players['Eficacia_goleadora'] = safe_divide(df_players['goal'], df_players['shot_on_target'])
    df_KPI_players['Habilidad_regate']= safe_divide(df_players['take_on_succeeded'], df_players['take_on'])
                                   
    df_KPI_players['influencia_finalización'] = safe_divide(df_players['final_third_area_pass_succeeded'], df_players['final_third_area_pass'])
    
    df_KPI_players['influencia_construccion'] = safe_divide(
        (df_players['middle_area_pass_succeeded'] + df_players['defensive_area_pass_succeeded']),
        (df_players['middle_area_pass'] + df_players['defensive_area_pass'])
    )
    df_KPI_players['precision_pases_largos'] = safe_divide(df_players['long_pass_succeeded'], df_players['long_pass'])
    df_KPI_players['verticalidad_pase'] = (
        ((df_players['forward_pass'] - (df_players['backward_pass'] + df_players['sideways_pass'])) * df_players['forward_pass_succeeded'])
    )
    
    df_KPI_players['talento'] = (((df_KPI_players['Habilidad_regate'] + df_players['shot_on_target']/df_players['total_shot']) + (df_players['control_under_pressure']) + ((df_players['free_kick'] +  df_players['corner_kick'] + df_players['penalty_kick'])) +
                                 (df_players['pass_succeeded']) + (df_players['goal'])+ (df_players['assist'] + df_players['key_pass']+ df_players['cross_succeeded']) +
                                 (df_players['take_on_succeeded']) + (df_players['tackle_succeeded']) + (df_players['aerial_duel_succeeded']) + (df_players['ground_duel_succeeded']) + 
                                 (df_players['loose_ball_duel_succeeded']) + (df_players['rating']) + (df_players['forward_pass_succeeded'])+ (df_players['save_by_catching'] + df_players['save_by_punching'])) - 
                                 (df_players['mistake'] + df_players['foul']+ df_players['own_goal']+ df_players['goal_conceded']+ df_players['offside']))

    df_KPI_players['efectividad_entradas'] = safe_divide(df_players['tackle_succeeded'], df_players['tackle'])
    df_KPI_players['efectividad_juego_aereo'] = safe_divide(df_players['aerial_duel_succeeded'], df_players['aerial_duel'])
    df_KPI_players['efectividad_duelos_suelo'] = safe_divide(df_players['ground_duel_succeeded'], df_players['ground_duel'])
    df_KPI_players['efectividad_balones_divididos'] = safe_divide(df_players['loose_ball_duel_succeeded'], df_players['loose_ball_duel'])
    df_KPI_players['contundencia'] = (df_KPI_players['efectividad_balones_divididos'] + 
                                      df_KPI_players['efectividad_duelos_suelo'] + 
                                      df_KPI_players['efectividad_juego_aereo'] + 
                                      df_KPI_players['efectividad_entradas']).round(2)
    df_KPI_players['capacidad_defensiva'] = safe_divide((df_players['intercept'] + df_players['clearance']+ df_players['tackle'] + df_players['block']+ df_players['ground_duel']+ df_players['loose_ball_duel'] +
                                                         df_players['recovery'] + df_players['save_by_catching'] + df_players['save_by_punching']), df_players['intervention'])
    df_KPI_players['capacidad_juego_aereo_def'] = safe_divide(safe_divide(df_players['aerial_clearance_succeeded'], df_players['aerial_clearance']), df_players['clearance'])
    df_KPI_players['rendimiento_posicional_def'] = safe_divide(df_players['defensive_line_support_succeeded'], df_players['defensive_line_support'])
    
    df_KPI_players['calificacion_Bepro'] = df_players['rating'].round(2)
    df_KPI_players['errores'] = df_players['mistake'].round(2)
    df_KPI_players['faltas'] = df_players['foul'].round(2)
    df_KPI_players['fuera_juego'] = df_players['offside'].round(2)
    df_KPI_players['tarj_amarilla'] = df_players['yellow_card'].round(2)
    df_KPI_players['tarj_roja'] = df_players['red_card'].round(2)
    df_KPI_players['gol_en_propia'] = df_players['own_goal'].round(2)
    df_KPI_players['gol_en_contra'] = df_players['goal_conceded'].round(2)
    
    # Redondear todas las columnas numéricas a 2 decimales
    df_KPI_players = df_KPI_players.round(2)
    
    print("df_KPI_players creado exitosamente dentro de la función.")
    return df_KPI_players

if 'df_players' not in globals():
    print("Error: 'df_players' no está definido. Asegúrate de cargar tu DataFrame antes de ejecutar esta función.")
else:
    try:
        print("Intentando crear df_KPI_players...")
        df_KPI_players = create_kpi_players(df_players)
        print("df_KPI_players creado exitosamente.")
        print("\nPrimeras filas de df_KPI_players:")
        print(df_KPI_players.head())
        print("\nInformación de df_KPI_players:")
        print(df_KPI_players.info())
    except ValueError as e:
        print(f"\nError al crear df_KPI_players: {e}")
        print("Por favor, verifica las columnas en df_players y asegúrate de que todas las columnas necesarias estén presentes.")
    except Exception as e:
        print(f"\nOcurrió un error inesperado: {e}")

if 'df_KPI_players' in globals():
    print("\ndf_KPI_players existe en el espacio de trabajo global")
    print(f"Forma de df_KPI_players: {df_KPI_players.shape}")
else:
    print("\ndf_KPI_players no se creó correctamente o no está en el espacio global.")
    print("Asegúrate de que no hay errores en la ejecución de la función y que 'df_players' contiene los datos correctos.")


In [ ]:


def calculate_kpis(df):
    # Verificar las columnas existentes
    existing_columns = df.columns.tolist()
    
    # Función de normalización (1 9)
    def normalize(data):
        min_val = np.min(data)
        max_val = np.max(data)
        if min_val == max_val:
            return np.full_like(data, 0.1)
        normalized = 0.1 + 1 * (data - min_val) / (max_val - min_val)
        return np.clip(normalized, 0.1, 1).round(2)

    kpi_columns = []

    # KPI 1: Condicional (Participacion y parametros fisicos)
    if all(col in existing_columns for col in ['participacion', 'capacidad_defensiva',
       'capacidad_juego_aereo_def']):
        condicional = (df['participacion']* 0.4+
                        df['capacidad_defensiva'] *0.3+ 
                        df['capacidad_juego_aereo_def'] * 0.3)
        df['KPI_condicional'] = (normalize(condicional) * 9.5).round(2)
        kpi_columns.append('KPI_condicional')

    # KPI 2: Construcción de Ataque
    if all(col in existing_columns for col in ['influencia_construccion', 'precision_pase',  'precision_pases_largos', 'verticalidad_pase', 'participacion']):
        construccion_ataque = ((df['influencia_construccion'] * 0.4 +
                               df['precision_pase']* 0.2 +
                               df['precision_pases_largos'] * 0.2 +
                               df['verticalidad_pase'] * 0.2) * df['participacion'])
        df['KPI_construccion_ataque'] = (normalize(construccion_ataque) * 9.5).round(2)
        kpi_columns.append('KPI_construccion_ataque')

    # KPI 3: Progresión
    if all(col in existing_columns for col in ['verticalidad_pase', 'precision_pase', 'precision_pases_largos', 'participacion']):
        progresion = ((df['verticalidad_pase'] * 0.6 +
                      df['precision_pase']* 0.2 +
                      df['precision_pases_largos'] * 0.2) * df['participacion'])
        df['KPI_progresion'] = (normalize(progresion) * 9.5).round(2)
        kpi_columns.append('KPI_progresion')

    # KPI 4: Habilidad Individual
    if all(col in existing_columns for col in ['talento', 'controles_bajo_presion', 'participacion']):
        habilidad_individual = ((df['talento'] * 0.8) + 
                              (df['controles_bajo_presion']) * 0.2 * df['participacion'])
        df['KPI_habilidad_individual'] = (normalize(habilidad_individual) * 9.5).round(2)
        kpi_columns.append('KPI_habilidad_individual')

    # KPI 5: Peligro Generado
    if all(col in existing_columns for col in ['influencia_pases_gol', 'participacion', 'Capacidad_disparos']):
        peligro_generado = ((df['influencia_pases_gol'] * 0.6 )+
                            (df['Capacidad_disparos'] * 0.4) * df['participacion'])
        df['KPI_peligro_generado'] = (normalize(peligro_generado)  * 9.5).round(2)
        kpi_columns.append('KPI_peligro_generado')

    # KPI 6: Finalización
    if all(col in existing_columns for col in ['Eficacia_disparos', 'Eficacia_goleadora', 'influencia_finalización', 'participacion']):
        finalizacion = (((df['Eficacia_disparos'] + df['Eficacia_goleadora'])* 0.6 +
                        df['influencia_finalización'] * 0.4) * df['participacion'])
        df['KPI_finalizacion'] = (normalize(finalizacion) * 9.5).round(2)
        kpi_columns.append('KPI_finalizacion') 

    # KPI 7: Eficacia Defensiva
    if all(col in existing_columns for col in ['efectividad_entradas', 'capacidad_defensiva', 'rendimiento_posicional_def', 'participacion', 'faltas']):
        eficacia_defensiva = (((df['efectividad_entradas'] - df['faltas']) * 0.3 +
                              df['capacidad_defensiva'] * 0.4 +
                              df['rendimiento_posicional_def'] * 0.3) * df['participacion'])
        df['KPI_eficacia_defensiva'] = (normalize(eficacia_defensiva) * 9.5).round(2)
        kpi_columns.append('KPI_eficacia_defensiva') 

    # KPI 8: Juego Aéreo
    if all(col in existing_columns for col in ['efectividad_juego_aereo', 'capacidad_juego_aereo_def', 'participacion']):
        juego_aereo = ((df['efectividad_juego_aereo'] * 0.7 +
                       df['capacidad_juego_aereo_def'] * 0.3) * df['participacion']) 
        df['KPI_juego_aereo'] = (normalize(juego_aereo) * 9.5).round(2)
        kpi_columns.append('KPI_juego_aereo')  

    # KPI 9: Capacidad de Recuperación
    if all(col in existing_columns for col in ['capacidad_defensiva', 'efectividad_balones_divididos', 'participacion']):
        capacidad_recuperacion = ((df['capacidad_defensiva'] * 0.7 +
                                  df['efectividad_balones_divididos'] * 0.3) * df['participacion'])
        df['KPI_capacidad_recuperacion'] = (normalize(capacidad_recuperacion) * 9.5).round(2)
        kpi_columns.append('KPI_capacidad_recuperacion')

    # KPI 10: Posicionamiento Táctico
    if all(col in existing_columns for col in ['rendimiento_posicional_def', 'fuera_juego', 'capacidad_defensiva', 'participacion']):
        posicionamiento_tactico = ((df['rendimiento_posicional_def'] * 0.5 +
                                   (1 - df['fuera_juego'] / df['participacion'].replace(0, np.nan)) * 0.3 +
                                   df['capacidad_defensiva'] * 0.2) * df['participacion']) 
        df['KPI_posicionamiento_tactico'] = (normalize(posicionamiento_tactico)  * 9.5).round(2)
        kpi_columns.append('KPI_posicionamiento_tactico')

    # KPI Rendimiento (suma promedio de los KPIs calculados, previa ponderacion por posicion)
    if kpi_columns:
        df['KPI_rendimiento'] = df[kpi_columns].mean(axis=1).round(2)
        kpi_columns.append('KPI_rendimiento')

    return df

# Aplicar la función al DataFrame
df_KPI_players = calculate_kpis(df_KPI_players)

# Verificar las columnas KPI que se calcularon
kpi_columns = [col for col in df_KPI_players.columns if col.startswith('KPI_')]

if kpi_columns:
    # Crear un ranking de los 20 mejores jugadores basado en KPI_rendimiento
    if 'KPI_rendimiento' in kpi_columns:
        top_20_players = df_KPI_players.nlargest(20, 'KPI_rendimiento')
        
        # Imprimir el ranking
        print("Ranking de los 20 mejores jugadores basado en KPI_rendimiento:")
        columns_to_show = ['player', 'team_name', 'main_position', 'league_name',
       'season_name', 'round_name', 'match'] + kpi_columns
        print(top_20_players[columns_to_show].to_string(index=False))

        # Mostrar estadísticas descriptivas de los KPIs
        print("\nEstadísticas descriptivas de los KPIs:")
        print(df_KPI_players[kpi_columns].describe())

        # Verificar si hay algún NaN en los resultados
        print("\nNúmero de NaN en cada KPI:")
        print(df_KPI_players[kpi_columns].isna().sum())
        
    else:
        print("KPI_rendimiento no se pudo calcular.")
else:
    print("No se pudieron calcular KPIs debido a la falta de columnas necesarias.")

# Imprimir las columnas disponibles en el DataFrame para diagnóstico
print("\nColumnas disponibles en el DataFrame:")

print(df_KPI_players.columns.tolist())


In [ ]:
# Verificar las columnas KPI que se calcularon
kpi_columns = [col for col in df_KPI_players.columns if col.startswith('KPI_')]

kpi_columns

In [ ]:
df_KPI_players.fillna(0, inplace=True)
df_KPI_players  =df_KPI_players[(df_KPI_players['main_position'] != 0) & (df_KPI_players['player'] != '0 0')]

In [ ]:
df_KPI_players.sort_values(by='KPI_rendimiento', ascending=False)

In [ ]:
# Verifica las posiciones únicas en la columna correspondiente
posiciones_unicas = df_KPI_players['main_position'].unique()
print(posiciones_unicas)

# Definir todas las posiciones posibles
#main_position= ['GK', 'CF', 'CM', 'RW', 'LB', 'RB', 'CB', 'CAM', 'LW', 'CDM', 'RM', 'LM', 'LWB', 'RWB', 'LF', 'RF']

# Elegir una posición (puedes cambiar este valor sin modificar el resto del código)
#main_position = 'CAM'  # Cambia este valor para elegir una posición distinta


In [ ]:

# Filtrar el DataFrame eliminando las filas donde 'main_position' y 'player' es igual a 0
#df_KPI_players.copy

#  Dataframe base df_KPI_players (todos los jugadores/todos los equipos)

In [ ]:
# Primero convertimos back_number a tipo string
df_KPI_players['back_number'] = df_KPI_players['back_number'].astype(str)

# Convertimos columnas numéricas potencialmente problemáticas
columnas_numericas = ['KPI_condicional', 'KPI_construccion_ataque', 'KPI_progresion', 
                     'KPI_habilidad_individual', 'KPI_peligro_generado', 'KPI_finalizacion', 
                     'KPI_eficacia_defensiva', 'KPI_juego_aereo', 'KPI_capacidad_recuperacion', 
                     'KPI_posicionamiento_tactico', 'KPI_rendimiento', 'play_time_minutes']

# Convertir columnas numéricas a float
for col in columnas_numericas:
    df_KPI_players[col] = pd.to_numeric(df_KPI_players[col], errors='coerce')

# Convertir columnas categóricas a string
columnas_categoricas = ['player', 'player_id', 'main_position', 'team_name', 'league_name', 
                       'season_name', 'round_name', 'match', 'match_id', 'home_away', 'starter_sub']
for col in columnas_categoricas:
    df_KPI_players[col] = df_KPI_players[col].astype(str)

# Guardar en formato parquet
df_KPI_players.to_parquet('../academyApp/data/archivos_parquet/df_KPI_players.parquet', index=False)

### Dataframe base df_KPI_players_position (añadimos ponderacion de KPI por posicion,todos los jugadores/todos los equipos)

In [ ]:
# Definir todas las posiciones posibles
main_position = ['GK', 'CF', 'CM', 'RW', 'LB', 'RB', 'CB', 'CAM', 'LW', 'CDM', 'RM', 'LM', 'LWB', 'RWB', 'LF', 'RF']

# Definir las ponderaciones para cada posición

pesos_main_position = {
    'GK': {'KPI_condicional': 0.6, 'KPI_construccion_ataque': 0.2, 'KPI_progresion': 0.05,
           'KPI_habilidad_individual': 0.1, 'KPI_peligro_generado': 0.05, 'KPI_finalizacion': 0.05,
           'KPI_eficacia_defensiva': 0.8, 'KPI_juego_aereo': 0.7, 'KPI_capacidad_recuperacion': 0.5,
           'KPI_posicionamiento_tactico': 0.6},
    
    'CB': {'KPI_condicional': 0.6, 'KPI_construccion_ataque': 0.3, 'KPI_progresion': 0.15,
           'KPI_habilidad_individual': 0.15, 'KPI_peligro_generado': 0.1, 'KPI_finalizacion': 0.1,
           'KPI_eficacia_defensiva': 0.9, 'KPI_juego_aereo': 0.75, 'KPI_capacidad_recuperacion': 0.7,
           'KPI_posicionamiento_tactico': 0.85},

    'RB': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.4, 'KPI_progresion': 0.55,
           'KPI_habilidad_individual': 0.6, 'KPI_peligro_generado': 0.35, 'KPI_finalizacion': 0.2,
           'KPI_eficacia_defensiva': 0.45, 'KPI_juego_aereo': 0.3, 'KPI_capacidad_recuperacion': 0.6,
           'KPI_posicionamiento_tactico': 0.5},

    'LB': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.4, 'KPI_progresion': 0.55,
           'KPI_habilidad_individual': 0.6, 'KPI_peligro_generado': 0.35, 'KPI_finalizacion': 0.2,
           'KPI_eficacia_defensiva': 0.45, 'KPI_juego_aereo': 0.3, 'KPI_capacidad_recuperacion': 0.6,
           'KPI_posicionamiento_tactico': 0.5},

    'RWB': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.75,
            'KPI_habilidad_individual': 0.65, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.25,
            'KPI_eficacia_defensiva': 0.45, 'KPI_juego_aereo': 0.3, 'KPI_capacidad_recuperacion': 0.6,
            'KPI_posicionamiento_tactico': 0.5},

    'LWB': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.75,
            'KPI_habilidad_individual': 0.65, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.25,
            'KPI_eficacia_defensiva': 0.45, 'KPI_juego_aereo': 0.3, 'KPI_capacidad_recuperacion': 0.6,
            'KPI_posicionamiento_tactico': 0.5},

    'CDM': {'KPI_condicional': 0.6, 'KPI_construccion_ataque': 0.55, 'KPI_progresion': 0.65,
            'KPI_habilidad_individual': 0.55, 'KPI_peligro_generado': 0.4, 'KPI_finalizacion': 0.2,
            'KPI_eficacia_defensiva': 0.75, 'KPI_juego_aereo': 0.6, 'KPI_capacidad_recuperacion': 0.75,
            'KPI_posicionamiento_tactico': 0.8},

    'CM': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.7, 'KPI_progresion': 0.7,
           'KPI_habilidad_individual': 0.65, 'KPI_peligro_generado': 0.55, 'KPI_finalizacion': 0.3,
           'KPI_eficacia_defensiva': 0.4, 'KPI_juego_aereo': 0.3, 'KPI_capacidad_recuperacion': 0.5,
           'KPI_posicionamiento_tactico': 0.5},

    'CAM': {'KPI_condicional': 0.4, 'KPI_construccion_ataque': 0.75, 'KPI_progresion': 0.7,
            'KPI_habilidad_individual': 0.85, 'KPI_peligro_generado': 0.8, 'KPI_finalizacion': 0.6,
            'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.2,
            'KPI_posicionamiento_tactico': 0.5},

    'RM': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'LM': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'RW': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'LW': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'LF': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'RF': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'CF': {'KPI_condicional': 0.6, 'KPI_construccion_ataque': 0.5, 'KPI_progresion': 0.5,
           'KPI_habilidad_individual': 0.8, 'KPI_peligro_generado': 0.85, 'KPI_finalizacion': 0.9,
           'KPI_eficacia_defensiva': 0.15, 'KPI_juego_aereo': 0.5, 'KPI_capacidad_recuperacion': 0.2,
           'KPI_posicionamiento_tactico': 0.4}
}


# Leer el archivo CSV (ajusta el nombre del archivo según sea necesario)
df_KPI_players = pd.read_parquet('../academyApp/data/archivos_parquet/df_KPI_players.parquet')

# Normalizar KPIs
kpi_columns = ['KPI_condicional', 'KPI_construccion_ataque', 'KPI_progresion', 
               'KPI_habilidad_individual', 'KPI_peligro_generado', 'KPI_finalizacion', 
               'KPI_eficacia_defensiva', 'KPI_juego_aereo', 'KPI_capacidad_recuperacion', 
               'KPI_posicionamiento_tactico']

# Normalizar los KPI a un rango de 0 a 9.5
df_KPI_players[kpi_columns] = df_KPI_players[kpi_columns].apply(
    lambda x: (x - x.min()) / (x.max() - x.min()) * 9.5)

# Calcular el rendimiento ponderado total según la posición de cada jugador
def calcular_rendimiento(row):
    posicion = row['main_position']
    peso = pesos_main_position.get(posicion, {})  # Obtén los pesos según la posición del jugador
    rendimiento_total = sum(row[kpi] * peso.get(kpi, 0) for kpi in kpi_columns)  # Aplica el peso para cada KPI
    return rendimiento_total


df_KPI_players['KPI_rendimiento'] = df_KPI_players[kpi_columns].mean(axis=1)

# Seleccionar columnas finales y ordenar
df_KPI_players_position = df_KPI_players[['back_number', 'player', 'player_id', 'main_position', 
                                       'team_name', 'league_name', 'season_name', 'round_name', 
                                       'match', 'match_id', 'home_away', 'starter_sub', 
                                       'play_time_minutes'] + kpi_columns + ['KPI_rendimiento']]

df_KPI_players_position = df_KPI_players_position.sort_values(by='KPI_rendimiento', ascending=False).round(2)

#Guardamos el DataFrame e como un archivo CSV
df_KPI_players_position.to_parquet('../academyApp/data/archivos_parquet/df_KPI_players_position.parquet', index=False)  
# Mostrar el DataFrame final ordenado
df_KPI_players_position.head(25)


### Obtenemos valores muy parecidos en ambos df lo que nos indica que la ponderacion de metricas ha sido acertada con una aproximacion del 99,2%

## KPIs Posicion & Metricas Avanzadas,  para ver los mejores "Rendimiento General, (todos jug./todos equipos)"


In [ ]:

# Definir todas las posiciones posibles
main_position = ['GK', 'CF', 'CM', 'RW', 'LB', 'RB', 'CB', 'CAM', 'LW', 'CDM', 'RM', 'LM', 'LWB', 'RWB', 'LF', 'RF']

# Definir las ponderaciones para cada posición

pesos_main_position = {
    'GK': {'KPI_condicional': 0.6, 'KPI_construccion_ataque': 0.2, 'KPI_progresion': 0.05,
           'KPI_habilidad_individual': 0.1, 'KPI_peligro_generado': 0.05, 'KPI_finalizacion': 0.05,
           'KPI_eficacia_defensiva': 0.8, 'KPI_juego_aereo': 0.7, 'KPI_capacidad_recuperacion': 0.5,
           'KPI_posicionamiento_tactico': 0.6},
    
    'CB': {'KPI_condicional': 0.6, 'KPI_construccion_ataque': 0.3, 'KPI_progresion': 0.15,
           'KPI_habilidad_individual': 0.15, 'KPI_peligro_generado': 0.1, 'KPI_finalizacion': 0.1,
           'KPI_eficacia_defensiva': 0.9, 'KPI_juego_aereo': 0.75, 'KPI_capacidad_recuperacion': 0.7,
           'KPI_posicionamiento_tactico': 0.85},

    'RB': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.4, 'KPI_progresion': 0.55,
           'KPI_habilidad_individual': 0.6, 'KPI_peligro_generado': 0.35, 'KPI_finalizacion': 0.2,
           'KPI_eficacia_defensiva': 0.45, 'KPI_juego_aereo': 0.3, 'KPI_capacidad_recuperacion': 0.6,
           'KPI_posicionamiento_tactico': 0.5},

    'LB': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.4, 'KPI_progresion': 0.55,
           'KPI_habilidad_individual': 0.6, 'KPI_peligro_generado': 0.35, 'KPI_finalizacion': 0.2,
           'KPI_eficacia_defensiva': 0.45, 'KPI_juego_aereo': 0.3, 'KPI_capacidad_recuperacion': 0.6,
           'KPI_posicionamiento_tactico': 0.5},

    'RWB': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.75,
            'KPI_habilidad_individual': 0.65, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.25,
            'KPI_eficacia_defensiva': 0.45, 'KPI_juego_aereo': 0.3, 'KPI_capacidad_recuperacion': 0.6,
            'KPI_posicionamiento_tactico': 0.5},

    'LWB': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.75,
            'KPI_habilidad_individual': 0.65, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.25,
            'KPI_eficacia_defensiva': 0.45, 'KPI_juego_aereo': 0.3, 'KPI_capacidad_recuperacion': 0.6,
            'KPI_posicionamiento_tactico': 0.5},

    'CDM': {'KPI_condicional': 0.6, 'KPI_construccion_ataque': 0.55, 'KPI_progresion': 0.65,
            'KPI_habilidad_individual': 0.55, 'KPI_peligro_generado': 0.4, 'KPI_finalizacion': 0.2,
            'KPI_eficacia_defensiva': 0.75, 'KPI_juego_aereo': 0.6, 'KPI_capacidad_recuperacion': 0.75,
            'KPI_posicionamiento_tactico': 0.8},

    'CM': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.7, 'KPI_progresion': 0.7,
           'KPI_habilidad_individual': 0.65, 'KPI_peligro_generado': 0.55, 'KPI_finalizacion': 0.3,
           'KPI_eficacia_defensiva': 0.4, 'KPI_juego_aereo': 0.3, 'KPI_capacidad_recuperacion': 0.5,
           'KPI_posicionamiento_tactico': 0.5},

    'CAM': {'KPI_condicional': 0.4, 'KPI_construccion_ataque': 0.75, 'KPI_progresion': 0.7,
            'KPI_habilidad_individual': 0.85, 'KPI_peligro_generado': 0.8, 'KPI_finalizacion': 0.6,
            'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.2,
            'KPI_posicionamiento_tactico': 0.5},

    'RM': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'LM': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'RW': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'LW': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'LF': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'RF': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'CF': {'KPI_condicional': 0.6, 'KPI_construccion_ataque': 0.5, 'KPI_progresion': 0.5,
           'KPI_habilidad_individual': 0.8, 'KPI_peligro_generado': 0.85, 'KPI_finalizacion': 0.9,
           'KPI_eficacia_defensiva': 0.15, 'KPI_juego_aereo': 0.5, 'KPI_capacidad_recuperacion': 0.2,
           'KPI_posicionamiento_tactico': 0.4}
}


# Leer el archivo CSV (ajusta el nombre del archivo según sea necesario)
df_KPI_players = pd.read_parquet('../academyApp/data/archivos_parquet/df_KPI_players.parquet')

# Elegir la posición
posicion_elegida = "LB"  # Aquí puedes definir la posición directamente

# Filtrar el DataFrame por la posición elegida
df_KPI_players_posicion = df_KPI_players[df_KPI_players['main_position'] == posicion_elegida]

# Normalizar KPIs
kpi_columns = ['KPI_condicional', 'KPI_construccion_ataque', 'KPI_progresion', 
               'KPI_habilidad_individual', 'KPI_peligro_generado', 'KPI_finalizacion', 
               'KPI_eficacia_defensiva', 'KPI_juego_aereo', 'KPI_capacidad_recuperacion', 
               'KPI_posicionamiento_tactico']

# Normalizar los KPI a un rango de 0 a 9.5
df_KPI_players_posicion[kpi_columns] = df_KPI_players_posicion[kpi_columns].apply(
    lambda x: (x - x.min()) / (x.max() - x.min()) * 9.5)

# Calcular el rendimiento ponderado total
def calcular_rendimiento(row):
    peso = pesos_main_position[posicion_elegida]
    rendimiento_total = sum(row[kpi] * peso[kpi] for kpi in kpi_columns)
    return rendimiento_total

df_KPI_players_posicion['KPI_rendimiento'] = df_KPI_players_posicion.apply(calcular_rendimiento, axis=1)
df_KPI_players_posicion['KPI_rendimiento'] = df_KPI_players_posicion[kpi_columns].mean(axis=1)

# Seleccionar columnas finales
df_KPI_players = df_KPI_players_posicion[['back_number', 'player', 'player_id', 'main_position', 
                                           'team_name', 'league_name', 'season_name', 'round_name', 
                                           'match', 'match_id', 'home_away', 'starter_sub', 
                                           'play_time_minutes', 'KPI_condicional', 
                                           'KPI_construccion_ataque', 'KPI_progresion', 
                                           'KPI_habilidad_individual', 'KPI_peligro_generado', 
                                           'KPI_finalizacion', 'KPI_eficacia_defensiva', 
                                           'KPI_juego_aereo', 'KPI_capacidad_recuperacion', 
                                           'KPI_posicionamiento_tactico', 'KPI_rendimiento']]

# Mostrar el DataFrame final
df_KPI_players_posicion = df_KPI_players.sort_values(by='KPI_rendimiento', ascending=False).round(2)
df_KPI_players_posicion

## Escalar de (0 a 9.5) para ver los mejores por posicion  "Rendimiento de la muestra" 

In [ ]:

# Definir todas las posiciones posibles
main_position = ['GK', 'CF', 'CM', 'RW', 'LB', 'RB', 'CB', 'CAM', 'LW', 'CDM', 'RM', 'LM', 'LWB', 'RWB', 'LF', 'RF', 'FW']


# Definir las ponderaciones para cada posición

pesos_main_position = {
    'GK': {'KPI_condicional': 0.6, 'KPI_construccion_ataque': 0.2, 'KPI_progresion': 0.05,
           'KPI_habilidad_individual': 0.1, 'KPI_peligro_generado': 0.05, 'KPI_finalizacion': 0.05,
           'KPI_eficacia_defensiva': 0.8, 'KPI_juego_aereo': 0.7, 'KPI_capacidad_recuperacion': 0.5,
           'KPI_posicionamiento_tactico': 0.6},
    
    'CB': {'KPI_condicional': 0.6, 'KPI_construccion_ataque': 0.3, 'KPI_progresion': 0.15,
           'KPI_habilidad_individual': 0.15, 'KPI_peligro_generado': 0.1, 'KPI_finalizacion': 0.1,
           'KPI_eficacia_defensiva': 0.9, 'KPI_juego_aereo': 0.75, 'KPI_capacidad_recuperacion': 0.7,
           'KPI_posicionamiento_tactico': 0.85},

    'RB': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.4, 'KPI_progresion': 0.55,
           'KPI_habilidad_individual': 0.6, 'KPI_peligro_generado': 0.35, 'KPI_finalizacion': 0.2,
           'KPI_eficacia_defensiva': 0.45, 'KPI_juego_aereo': 0.3, 'KPI_capacidad_recuperacion': 0.6,
           'KPI_posicionamiento_tactico': 0.5},

    'LB': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.4, 'KPI_progresion': 0.55,
           'KPI_habilidad_individual': 0.6, 'KPI_peligro_generado': 0.35, 'KPI_finalizacion': 0.2,
           'KPI_eficacia_defensiva': 0.45, 'KPI_juego_aereo': 0.3, 'KPI_capacidad_recuperacion': 0.6,
           'KPI_posicionamiento_tactico': 0.5},

    'RWB': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.75,
            'KPI_habilidad_individual': 0.65, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.25,
            'KPI_eficacia_defensiva': 0.45, 'KPI_juego_aereo': 0.3, 'KPI_capacidad_recuperacion': 0.6,
            'KPI_posicionamiento_tactico': 0.5},

    'LWB': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.75,
            'KPI_habilidad_individual': 0.65, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.25,
            'KPI_eficacia_defensiva': 0.45, 'KPI_juego_aereo': 0.3, 'KPI_capacidad_recuperacion': 0.6,
            'KPI_posicionamiento_tactico': 0.5},

    'CDM': {'KPI_condicional': 0.6, 'KPI_construccion_ataque': 0.55, 'KPI_progresion': 0.65,
            'KPI_habilidad_individual': 0.55, 'KPI_peligro_generado': 0.4, 'KPI_finalizacion': 0.2,
            'KPI_eficacia_defensiva': 0.75, 'KPI_juego_aereo': 0.6, 'KPI_capacidad_recuperacion': 0.75,
            'KPI_posicionamiento_tactico': 0.8},

    'CM': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.7, 'KPI_progresion': 0.7,
           'KPI_habilidad_individual': 0.65, 'KPI_peligro_generado': 0.55, 'KPI_finalizacion': 0.3,
           'KPI_eficacia_defensiva': 0.4, 'KPI_juego_aereo': 0.3, 'KPI_capacidad_recuperacion': 0.5,
           'KPI_posicionamiento_tactico': 0.5},

    'CAM': {'KPI_condicional': 0.4, 'KPI_construccion_ataque': 0.75, 'KPI_progresion': 0.7,
            'KPI_habilidad_individual': 0.85, 'KPI_peligro_generado': 0.8, 'KPI_finalizacion': 0.6,
            'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.2,
            'KPI_posicionamiento_tactico': 0.5},

    'RM': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'LM': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'RW': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'LW': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'LF': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'RF': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'CF': {'KPI_condicional': 0.6, 'KPI_construccion_ataque': 0.5, 'KPI_progresion': 0.5,
           'KPI_habilidad_individual': 0.8, 'KPI_peligro_generado': 0.85, 'KPI_finalizacion': 0.9,
           'KPI_eficacia_defensiva': 0.15, 'KPI_juego_aereo': 0.5, 'KPI_capacidad_recuperacion': 0.2,
           'KPI_posicionamiento_tactico': 0.4}
}


# Leer el archivo CSV (ajusta el nombre del archivo según sea necesario)
df_KPI_players = pd.read_parquet('../academyApp/data/archivos_parquet/df_KPI_players.parquet')

# Calcular el rendimiento ponderado total como el promedio de los KPIs
def calcular_rendimiento_promedio(df):
    kpi_columns = [
        'KPI_condicional', 'KPI_construccion_ataque', 'KPI_progresion', 
        'KPI_habilidad_individual', 'KPI_peligro_generado', 'KPI_finalizacion', 
        'KPI_eficacia_defensiva', 'KPI_juego_aereo', 'KPI_capacidad_recuperacion', 
        'KPI_posicionamiento_tactico'
    ]
    # Calcular el promedio de los KPIs para cada jugador
    df['KPI_rendimiento'] = df[kpi_columns].mean(axis=1)

# Suponiendo que df_KPI_players es el DataFrame original con los datos de los jugadores
calcular_rendimiento_promedio(df_KPI_players)

# Normalizar los valores entre 0 y 9.5
def normalizar(valores):
    min_val = valores.min()
    max_val = valores.max()
    
    # Normalización al rango [0, 9.5]
    normalizado = ((valores - min_val) / (max_val - min_val)) * 9.5
    return normalizado

# Elegir la posición
posicion_elegida = input(f"Elija una posición de las siguientes opciones: {', '.join(main_position)}: ")
while posicion_elegida not in main_position:
    posicion_elegida = input("Posición inválida. Elija una posición válida: ")

# Filtrar el DataFrame por la posición elegida
df_KPI_players_posicion = df_KPI_players[df_KPI_players['main_position'] == posicion_elegida]

# Filtrar las columnas deseadas del DataFrame
df_KPI_players_posicion  = df_KPI_players_posicion[[ 
    'back_number', 'player', 'player_id', 'main_position', 'team_name', 
    'league_name', 'season_name', 'round_name', 'match', 'match_id',
    'home_away', 'starter_sub', 'play_time_minutes',
    'KPI_condicional', 'KPI_construccion_ataque',
    'KPI_progresion', 'KPI_habilidad_individual', 'KPI_peligro_generado', 
    'KPI_finalizacion', 'KPI_eficacia_defensiva', 'KPI_juego_aereo',
    'KPI_capacidad_recuperacion', 'KPI_posicionamiento_tactico', 
    'KPI_rendimiento'  # Asegúrate de incluir esta columna para normalizarla
]]

# Normalizar las columnas de KPI
kpi_columns = ['KPI_condicional', 'KPI_construccion_ataque', 'KPI_progresion', 
               'KPI_habilidad_individual', 'KPI_peligro_generado', 
               'KPI_finalizacion', 'KPI_eficacia_defensiva', 
               'KPI_juego_aereo', 'KPI_capacidad_recuperacion', 
               'KPI_posicionamiento_tactico', 'KPI_rendimiento']

for kpi in kpi_columns:
    df_KPI_players_posicion[kpi] = normalizar(df_KPI_players_posicion[kpi]).round(2)


# Mostrar el DataFrame final
df_KPI_players_normalizado= df_KPI_players_posicion.sort_values(by='KPI_rendimiento', ascending=False).round(2)
df_KPI_players_normalizado.head(25)

##  Dataframes de la cantera por partidos 

In [ ]:
#Partimos del dataframe de posisiciones creado
df_KPI_players_position = pd.read_parquet('../academyApp/data/archivos_parquet/df_KPI_players_position.parquet')
df_KPI_players_position


*dfs_Performance_per_match_all_positions*


In [ ]:
# Filtrar el DataFrame por Deportivo Alavés

df_Academia_alaves_matches= df_KPI_players_position[df_KPI_players_position['team_name'].str.contains('Deportivo Alavés', case=False)]
df_Academia_alaves_matches.head(50)


*dfs_Performance_per match, team, season & position*


In [ ]:

# Definir el filtro de temporada, equipo y posición
season_filter = '24/25'
team_filter = 'Deportivo Alavés B'
position_filter = 'CB'

kpi_columns = ['KPI_condicional', 'KPI_construccion_ataque', 'KPI_progresion', 
               'KPI_habilidad_individual', 'KPI_peligro_generado', 
               'KPI_finalizacion', 'KPI_eficacia_defensiva', 
               'KPI_juego_aereo', 'KPI_capacidad_recuperacion', 
               'KPI_posicionamiento_tactico', 'KPI_rendimiento']

# Filtrar el DataFrame original según los filtros establecidos
filtered_df_Academia_matches = df_Academia_alaves_matches.loc[
    (df_Academia_alaves_matches['season_name'] == season_filter) &
    (df_Academia_alaves_matches['team_name'].str.contains(team_filter, case=False)) &
    (df_Academia_alaves_matches['main_position'] == position_filter) 
]

# Crear una nueva columna temporal extrayendo el número de 'round_name'
filtered_df_Academia_matches['round_number'] = filtered_df_Academia_matches['round_name'].str.extract('(\d+)').fillna(0).astype(int)

# Ordenar el DataFrame por 'season_name' y luego por 'round_number'
filtered_df_Academia_matches = filtered_df_Academia_matches.sort_values(by=['season_name', 'round_number'], ascending=[True, True])

# Eliminar la columna temporal 'round_number'
filtered_df_Academia_matches = filtered_df_Academia_matches.drop(columns=['round_number'])

# Lista de columnas KPI para normalizar, excluyendo 'playtime_minutes' si está en la lista
kpi_columns = [col for col in kpi_columns if col != 'playtime_minutes']

# Función para normalizar entre 1 y 9
def normalizar(valores):
    min_val = valores.min()
    max_val = valores.max()
    # Evitar división por cero si todos los valores son iguales
    if min_val == max_val:
        return pd.Series([5] * len(valores))  # Asignar 5 si todos los valores son iguales
    else:
        return 1 + (valores - min_val) * (9 - 1) / (max_val - min_val)

# Aplicar normalización solo a las columnas KPI, excluyendo 'playtime_minutes'
for kpi in kpi_columns:
    filtered_df_Academia_matches[kpi] = normalizar(filtered_df_Academia_matches[kpi]).round(2)

# Mostrar las primeras filas para verificar que 'playtime_minutes' no se ha redondeado
filtered_df_Academia_matches.head(20)


In [ ]:
season_filter = '24/25'  # Temporada a filtrar:['21/22', '22/23', '23/24', '24/25']
team_filter = 'Deportivo Alavés B'  # Nombre del equipo a filtrar ['Deportivo Alavés Cad. A', 'Deportivo Alavés Juv. A', 'Deportivo Alavés C', 'Deportivo Alavés B' ]
position_filter = 'CB'  # Posición a filtrar: ['GK', 'CF', 'CM', 'RW', 'LB', 'RB', 'CB', 'CAM', 'LW', 'CDM', 'RM', 'LM', 'LWB', 'RWB', 'LF', 'RF']




filtered_df_Academia_matches = df_Academia_alaves_matches.loc[
    (df_Academia_alaves_matches['season_name'] == season_filter) &
    (df_Academia_alaves_matches['team_name'].str.contains(team_filter, case=False)) &
    (df_Academia_alaves_matches['main_position'] == position_filter) 
]
# Crear una nueva columna temporal extrayendo el número de 'round_name'
filtered_df_Academia_matches['round_number'] = filtered_df_Academia_matches['round_name'].str.extract('(\d+)').fillna(0).astype(int)

# Ordenar el dataframe por 'season_name' y luego por 'round_number'
filtered_df_Academia_matches = filtered_df_Academia_matches.sort_values(by=['season_name', 'round_number'], ascending=[True, True])

# Eliminar la columna temporal 'round_number'
filtered_df_Academia_matches = filtered_df_Academia_matches.drop(columns=['round_number'])

#Función para normalizar entre 1 y 9
def normalizar(valores):
    min_val = valores.min()
    max_val = valores.max()
    # Evitar división por cero si todos los valores son iguales
    if min_val == max_val:
        return pd.Series([5] * len(valores))  # Asignar 5 si todos los valores son iguales
    else:
        return 1 + (valores - min_val) * (9 - 1) / (max_val - min_val)

# Aplicar normalización a cada columna de KPI
for kpi in kpi_columns:
    filtered_df_Academia_matches[kpi] = normalizar(filtered_df_Academia_matches[kpi]).round(2)


filtered_df_Academia_matches.head(50)

In [ ]:


df = filtered_df_Academia_matches

# Convertir la columna 'round_name' a texto para que podamos aplicar .str.extract()
df['round_name'] = df['round_name'].astype(str)

# Extraer el número de jornada (asumiendo que está en formato 'Jornada X')
df['round_number'] = df['round_name'].str.extract('(\d+)').astype(float)

# Rellenar valores NaN en 'round_number' con 0 para aquellos jugadores que no tienen jornada
df['round_number'] = df['round_number'].fillna(0)

# Crear un índice de jugadores y jornadas para asegurarnos de que todos los jugadores tengan cada jornada
players = df['player'].unique()
rounds = df['round_name'].unique()
index = pd.MultiIndex.from_product([players, rounds], names=['player', 'round_name'])

# Reindexar el DataFrame para asegurar que cada jugador tenga cada jornada
df = df.set_index(['player', 'round_name']).reindex(index, fill_value=0).reset_index()

# Color de fondo para el gráfico y el plot
bg_color = '#0E3F5C'

# Generar el gráfico
fig, ax = plt.subplots(figsize=(14, 8), facecolor=bg_color)
ax.set_facecolor(bg_color)

# Graficar cada jugador con una línea más gruesa y contar jornadas jugadas
for player in df['player'].unique():
    player_data = df[df['player'] == player]
    
    # Contar jornadas jugadas (aquellas donde 'KPI_rendimiento' no sea 0)
    total_jornadas_jugadas = player_data['KPI_rendimiento'].ne(0).sum()
    
    # Crear la etiqueta del jugador con el total de jornadas jugadas
    label = f"{player} ({total_jornadas_jugadas})"
    
    # Graficar con líneas más gruesas
    ax.plot(player_data['round_name'], player_data['KPI_rendimiento'], label=label, linewidth=2)  # Aquí se cambia el grosor de las líneas

# Personalización del gráfico
ax.set_title('Evolución del KPI de Rendimiento centrales (Deportivo Alaves B) Temp:24/25', color='white')
ax.set_ylabel('KPI de Rendimiento', color='white')

# Ajustes de la leyenda
ax.legend(title="Jugadores (jornadas jugadas)", bbox_to_anchor=(1.05, 1), loc='upper left', facecolor=bg_color, framealpha=0, labelcolor='white', title_fontsize='medium')
ax.grid(True, linestyle='--', alpha=0.2, color='white')

# Cambiar el color de los ejes y etiquetas a blanco para mejor visibilidad en el fondo oscuro
ax.tick_params(colors='white')
ax.spines['top'].set_color(bg_color)
ax.spines['right'].set_color(bg_color)
ax.spines['left'].set_color(bg_color)
ax.spines['bottom'].set_color(bg_color)

plt.show()


In [ ]:

# Definir las columnas que quieres excluir del promedio
columnas_excluidas = ['back_number', 'player', 'player_id', 'main_position', 'team_name', 'league_name', 'season_name', 'round_name', 'match', 'match_id', 'home_away', 'starter_sub', 'play_time_minutes']

# Crear un nuevo DataFrame que contendrá todas las filas promedio por temporada
average_rows = []

# Iterar sobre cada temporada para calcular el promedio por temporada
for season in ['21/22', '22/23', '23/24', '24/25']:  # Cambia aquí si necesitas más temporadas
    # Filtrar el DataFrame original por la temporada actual
    df_season = filtered_df_Academia_matches[filtered_df_Academia_matches['season_name'] == season]
    
    # Calcular el promedio de las columnas KPI para esta temporada
    average = df_season.drop(columns=columnas_excluidas).mean()
    
    # Contar el número de jornadas únicas en esa temporada
    total_jornadas = df_season['round_name'].nunique()
    
    # Crear una nueva fila para esta temporada con el promedio y el total de jornadas en round_name
    average_label = f'Average{season[-2:]}'  
    average_row = pd.DataFrame({col: average[col] if col not in columnas_excluidas else average_label for col in filtered_df_Academia_matches.columns}, index=[0])
    
    # Asignar temporada y número total de jornadas en las columnas correspondientes
    average_row['season_name'] = season
    average_row['round_name'] = f'Jornadas {total_jornadas}'
    
    # Añadir la fila promedio al listado
    average_rows.append(average_row)

# Concatenar todas las filas promedio al DataFrame original
average_rows_df = pd.concat(average_rows, ignore_index=True)
df_KPI_playersavg_normalizado = pd.concat([filtered_df_Academia_matches, average_rows_df], ignore_index=True)

# Crear una nueva columna temporal extrayendo el número de 'round_name'
df_KPI_playersavg_normalizado['round_number'] = df_KPI_playersavg_normalizado['round_name'].str.extract('(\d+)').fillna(0).astype(int)

# Ordenar el dataframe por 'season_name' y luego por 'round_number'
df_KPI_playersavg_normalizado = df_KPI_playersavg_normalizado.sort_values(by=['season_name', 'round_number'], ascending=[True, True])

# Eliminar la columna temporal 'round_number'
df_KPI_playersavg_normalizado = df_KPI_playersavg_normalizado.drop(columns=['round_number'])

# Lista de columnas KPI para calcular promedios y normalizar (asegúrate de definir kpi_columns previamente)
kpi_columns = [col for col in df_KPI_playersavg_normalizado.columns if col not in columnas_excluidas]

# Función para normalizar entre 1 y 9
def normalizar(valores):
    min_val = valores.min()
    max_val = valores.max()
    # Evitar división por cero si todos los valores son iguales
    if min_val == max_val:
        return pd.Series([5] * len(valores))  # Asignar 5 si todos los valores son iguales
    else:
        return 1 + (valores - min_val) * (9 - 1) / (max_val - min_val)

# Aplicar normalización a cada columna de KPI
for kpi in kpi_columns:
    df_KPI_playersavg_normalizado[kpi] = normalizar(df_KPI_playersavg_normalizado[kpi]).round(2)

# Crear una lista para almacenar las filas de promedio por jugador y por temporada
average_players_rows = []

# Iterar sobre cada jugador y temporada para calcular el promedio
for player in df_KPI_playersavg_normalizado['player'].unique():
    if 'Average' not in player:  # Filtrar los promedios de temporada
        for season in df_KPI_playersavg_normalizado['season_name'].unique():
            # Filtrar el DataFrame para el jugador y temporada actual
            df_player_season = df_KPI_playersavg_normalizado[
                (df_KPI_playersavg_normalizado['player'] == player) &
                (df_KPI_playersavg_normalizado['season_name'] == season)
            ]
            
            # Si el jugador no tiene registros para esa temporada, continuar
            if df_player_season.empty:
                continue

            # Calcular el promedio de las columnas KPI para este jugador y temporada
            average = df_player_season[kpi_columns].mean()
            
            # Calcular la suma total de 'play_time_minutes' para el jugador en la temporada
            total_minutes = df_player_season['play_time_minutes'].sum()
            
            # Contar el número de jornadas jugadas por el jugador en la temporada
            jornadas_jugadas = df_player_season['round_name'].nunique()
            
            # Crear una nueva fila con el nombre del jugador seguido de ".avg" y número de jornadas jugadas
            average_label = f"{player}.avg"
            average_row = pd.DataFrame({col: average[col] if col in kpi_columns else average_label for col in df_KPI_playersavg_normalizado.columns}, index=[0])
            
            # Asignar el nombre del jugador, temporada, número de jornadas jugadas y total de minutos en las columnas correspondientes
            average_row['player'] = average_label
            average_row['season_name'] = season  
            average_row['round_name'] = f'Jornadas {jornadas_jugadas}'
            average_row['play_time_minutes'] = total_minutes  # Asignar la suma total de minutos
            
            # Añadir la fila promedio al listado
            average_players_rows.append(average_row)

# Concatenar todas las filas promedio de jugadores al DataFrame original
average_players_df = pd.concat(average_players_rows, ignore_index=True)
df_KPI_playersavg_normalizado = pd.concat([df_KPI_playersavg_normalizado, average_players_df], ignore_index=True)

# Mostrar el DataFrame actualizado con filas de promedio por jugador y temporada específica
df_KPI_playersavg_normalizado.head(50).round(2)


In [ ]:

# Definir las columnas que quieres excluir del promedio
columnas_excluidas = ['back_number', 'player', 'player_id', 'main_position', 'team_name', 'league_name', 'season_name', 'round_name', 'match', 'match_id', 'home_away', 'starter_sub', 'play_time_minutes']

# Crear un nuevo DataFrame que contendrá todas las filas promedio por temporada
average_rows = []

# Iterar sobre cada temporada para calcular el promedio por temporada
for season in ['24/25']:  # Cambia aquí si necesitas más temporadas
    # Filtrar el DataFrame original por la temporada actual
    df_season = filtered_df_Academia_matches[filtered_df_Academia_matches['season_name'] == season]
    
    # Calcular el promedio de las columnas KPI para esta temporada
    average = df_season.drop(columns=columnas_excluidas).mean()
    
    # Contar el número de jornadas únicas en esa temporada
    total_jornadas = df_season['round_name'].nunique()
    
    # Calcular el promedio de minutos jugados por jugador en la temporada
    avg_minutes_played = df_season['play_time_minutes'].mean()
    
    # Crear una nueva fila para esta temporada con el promedio y el total de jornadas en round_name
    average_label = f'Average{season[-2:]}'  
    average_row = pd.DataFrame({col: average[col] if col not in columnas_excluidas else average_label for col in filtered_df_Academia_matches.columns}, index=[0])
    
    # Asignar temporada y número total de jornadas en las columnas correspondientes
    average_row['season_name'] = season
    average_row['round_name'] = f'Jornadas {total_jornadas}'
    average_row['play_time_minutes'] = avg_minutes_played  # Asignar el promedio de minutos jugados
    
    # Añadir la fila promedio al listado
    average_rows.append(average_row)

# Concatenar todas las filas promedio al DataFrame original
average_rows_df = pd.concat(average_rows, ignore_index=True)
df_KPI_playersavg_normalizado = pd.concat([filtered_df_Academia_matches, average_rows_df], ignore_index=True)

# Crear una nueva columna temporal extrayendo el número de 'round_name'
df_KPI_playersavg_normalizado['round_number'] = df_KPI_playersavg_normalizado['round_name'].str.extract('(\d+)').fillna(0).astype(int)

# Ordenar el dataframe por 'season_name' y luego por 'round_number'
df_KPI_playersavg_normalizado = df_KPI_playersavg_normalizado.sort_values(by=['season_name', 'round_number'], ascending=[True, True])

# Eliminar la columna temporal 'round_number'
df_KPI_playersavg_normalizado = df_KPI_playersavg_normalizado.drop(columns=['round_number'])

# Lista de columnas KPI para calcular promedios y normalizar (asegúrate de definir kpi_columns previamente)
kpi_columns = [col for col in df_KPI_playersavg_normalizado.columns if col not in columnas_excluidas]

# Función para normalizar entre 1 y 9
def normalizar(valores):
    min_val = valores.min()
    max_val = valores.max()
    # Evitar división por cero si todos los valores son iguales
    if min_val == max_val:
        return pd.Series([5] * len(valores))  # Asignar 5 si todos los valores son iguales
    else:
        return 1 + (valores - min_val) * (9 - 1) / (max_val - min_val)

# Aplicar normalización a cada columna de KPI
for kpi in kpi_columns:
    df_KPI_playersavg_normalizado[kpi] = normalizar(df_KPI_playersavg_normalizado[kpi]).round(2)

# Crear una lista para almacenar las filas de promedio por jugador y por temporada
average_players_rows = []

# Iterar sobre cada jugador y temporada para calcular el promedio
for player in df_KPI_playersavg_normalizado['player'].unique():
    if 'Average' not in player:  # Filtrar los promedios de temporada
        for season in df_KPI_playersavg_normalizado['season_name'].unique():
            # Filtrar el DataFrame para el jugador y temporada actual
            df_player_season = df_KPI_playersavg_normalizado[
                (df_KPI_playersavg_normalizado['player'] == player) &
                (df_KPI_playersavg_normalizado['season_name'] == season)
            ]
            
            # Si el jugador no tiene registros para esa temporada, continuar
            if df_player_season.empty:
                continue

            # Calcular el promedio de las columnas KPI para este jugador y temporada
            average = df_player_season[kpi_columns].mean()
            
            # Calcular la suma total de 'play_time_minutes' para el jugador en la temporada
            total_minutes = df_player_season['play_time_minutes'].sum()
            
            # Contar el número de jornadas jugadas por el jugador en la temporada
            jornadas_jugadas = df_player_season['round_name'].nunique()
            
            # Crear una nueva fila con el nombre del jugador seguido de ".avg" y número de jornadas jugadas
            average_label = f"{player}.avg"
            average_row = pd.DataFrame({col: average[col] if col in kpi_columns else average_label for col in df_KPI_playersavg_normalizado.columns}, index=[0])
            
            # Asignar el nombre del jugador, temporada, número de jornadas jugadas y total de minutos en las columnas correspondientes
            average_row['player'] = average_label
            average_row['season_name'] = season  
            average_row['round_name'] = f'Jornadas {jornadas_jugadas}'
            average_row['play_time_minutes'] = total_minutes  # Asignar la suma total de minutos
            
            # Añadir la fila promedio al listado
            average_players_rows.append(average_row)

# Concatenar todas las filas promedio de jugadores al DataFrame original
average_players_df = pd.concat(average_players_rows, ignore_index=True)
df_KPI_playersavg_normalizado = pd.concat([df_KPI_playersavg_normalizado, average_players_df], ignore_index=True)

# Mostrar el DataFrame actualizado con filas de promedio por jugador y temporada específica
df_KPI_playersavg_normalizado.head(50).round(2)


In [ ]:

# Filtrar el DataFrame para obtener solo las filas promedio (por temporada y por jugador)
df_avg = df_KPI_playersavg_normalizado[
    df_KPI_playersavg_normalizado['season_name'].str.contains('Average') | 
    df_KPI_playersavg_normalizado['player'].str.contains('.avg')
]

# Mantener solo las columnas de KPIs y los identificadores de cada fila promedio
df_avg = df_avg[['player', 'season_name'] + kpi_columns]

# Derretir el DataFrame para convertirlo a formato largo (esto facilita la creación de gráficos de barras)
df_melted = df_avg.melt(id_vars=['player', 'season_name'], value_vars=kpi_columns, 
                        var_name='KPI', value_name='Value')

# Establecer un color de fondo para el gráfico
bg_color = '#0E3F5C'

# Crear el gráfico
plt.figure(figsize=(14, 8), facecolor=bg_color)
ax = plt.gca()
ax.set_facecolor(bg_color)

# Crear un gráfico de barras agrupadas por KPI y tipo de promedio (temporada o jugador)
for label, df_group in df_melted.groupby('player'):
    ax.bar(df_group['KPI'], df_group['Value'], label=label, alpha=0.7)

# Personalizar el gráfico
ax.set_title('Comparación de KPIs entre Promedios de Temporada y Promedios de Jugadores', color='white')
ax.set_ylabel('Valor Normalizado (1-9)', color='white')
ax.set_xlabel('KPI', color='white')
ax.legend(title="Centrales Dvo.Alaves B", bbox_to_anchor=(1.05, 1), loc='upper left', facecolor=bg_color, framealpha=0, labelcolor='white', title_fontsize='medium')
ax.grid(True, linestyle='--', alpha=0.5, color='white')

# Cambiar el color de los ejes y etiquetas a blanco para visibilidad en el fondo oscuro
ax.tick_params(colors='white')
ax.spines['top'].set_color(bg_color)
ax.spines['right'].set_color(bg_color)
ax.spines['left'].set_color(bg_color)
ax.spines['bottom'].set_color(bg_color)

# Mostrar el gráfico
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Identificar las columnas de los KPIs
kpi_columns = [
    'KPI_condicional', 'KPI_construccion_ataque', 'KPI_progresion',
    'KPI_habilidad_individual', 'KPI_peligro_generado', 'KPI_finalizacion',
    'KPI_eficacia_defensiva', 'KPI_juego_aereo', 'KPI_capacidad_recuperacion',
    'KPI_posicionamiento_tactico', 'KPI_rendimiento'
]

# Filtrar DataFrame para promedios
df_avg = df_KPI_playersavg_normalizado[
    df_KPI_playersavg_normalizado['player'].str.contains('.avg|Average')
]

# Calcular jornadas y minutos
df_avg['jornadas_jugadas'] = df_avg['round_name'].str.extract('(\d+)').astype(float)
df_avg['total_minutes'] = df_avg['play_time_minutes'].astype(float) * 10

# Seleccionar columnas necesarias
df_avg = df_avg[['player', 'season_name', 'jornadas_jugadas', 'total_minutes'] + kpi_columns]

# Convertir a formato largo
df_melted = df_avg.melt(
    id_vars=['player', 'season_name', 'jornadas_jugadas', 'total_minutes'],
    value_vars=kpi_columns,
    var_name='KPI',
    value_name='Value'
)

# Crear etiquetas
df_melted['label'] = df_melted.apply(
    lambda row: (f"{row['player']}\n({int(row['jornadas_jugadas'])} jornadas, "
                f"{row['total_minutes']:.1f} min)" if "Average" not in row['player']
                else f"Average Temporada {row['season_name']}\n"
                f"({int(row['jornadas_jugadas'])} jornadas)"),
    axis=1
)

# Configuración del gráfico
bg_color = '#0E3F5C'
plt.figure(figsize=(14, 8), facecolor=bg_color)
ax = plt.gca()
ax.set_facecolor(bg_color)

# Crear barras agrupadas
bar_width = 0.15
kpi_categories = df_melted['KPI'].unique()
x = np.arange(len(kpi_categories))

# Graficar grupos
for i, (label, df_group) in enumerate(df_melted.groupby('label')):
    ax.bar(x + i * bar_width, df_group['Value'], width=bar_width, label=label)

# Personalizar gráfico
ax.set_title('Comparación de KPIs Promedios Jugador y Promedio Temporada 24/25', color='white')
ax.set_ylabel('Valor (0-9)', color='white')
ax.set_xticks(x + bar_width * (len(df_melted['label'].unique()) - 1) / 2)
ax.set_xticklabels(kpi_categories, color='white', rotation=45)

# Configurar leyenda
legend = ax.legend(
    title="Centrales Dvo.Alaves B",
    bbox_to_anchor=(1.05, 1),
    loc='upper left',
    facecolor=bg_color,
    framealpha=0,
    labelcolor='white',
    title_fontsize='medium'
)
legend.get_title().set_color('#00FFFF')

# Configurar estilo
ax.grid(True, linestyle='--', alpha=0.5, color='white')
ax.tick_params(colors='white')
for spine in ax.spines.values():
    spine.set_color(bg_color)

plt.tight_layout()
plt.show()

In [ ]:
# Identificar las columnas de los KPIs
kpi_columns = [
    'KPI_condicional', 'KPI_construccion_ataque', 'KPI_progresion',
    'KPI_habilidad_individual', 'KPI_peligro_generado', 'KPI_finalizacion',
    'KPI_eficacia_defensiva', 'KPI_juego_aereo', 'KPI_capacidad_recuperacion',
    'KPI_posicionamiento_tactico', 'KPI_rendimiento'
]

# Filtrar DataFrame para promedios
df_avg = df_KPI_playersavg_normalizado[
    df_KPI_playersavg_normalizado['player'].str.contains('.avg|Average')
]

# Calcular jornadas y minutos
df_avg['jornadas_jugadas'] = df_avg['round_name'].str.extract('(\d+)').astype(float)
df_avg['total_minutes'] = df_avg['play_time_minutes'].astype(float) * 10

# Mantener columnas necesarias
df_avg = df_avg[['player', 'season_name', 'jornadas_jugadas', 'total_minutes'] + kpi_columns]

# Convertir a formato largo
df_melted = df_avg.melt(
    id_vars=['player', 'season_name', 'jornadas_jugadas', 'total_minutes'],
    value_vars=kpi_columns,
    var_name='KPI',
    value_name='Value'
)

# Crear etiquetas con formato LaTeX
df_melted['label'] = df_melted.apply(
    lambda row: f"$\\bf{{{row['player']}}}$\n"
                f"$\\mathrm{{({int(row['jornadas_jugadas'])} \\ Jornadas, \\ {row['total_minutes']:.1f} \\ min)}}$"
                if "Average" not in row['player']
                else f"$\\bf{{Average\\ Temporada\\ {row['season_name']}}}$\n"
                f"$\\mathrm{{({int(row['jornadas_jugadas'])} \\ Jornadas)}}$",
    axis=1
)

# Configuración del gráfico
bg_color = '#0E3F5C'
plt.figure(figsize=(14, 8), facecolor=bg_color)
ax = plt.gca()
ax.set_facecolor(bg_color)

# Crear barras agrupadas
bar_width = 0.15
kpi_categories = df_melted['KPI'].unique()
x = np.arange(len(kpi_categories))

# Graficar grupos
for i, (label, df_group) in enumerate(df_melted.groupby('label')):
    ax.bar(x + i * bar_width, df_group['Value'], width=bar_width, label=label)

# Personalizar gráfico
ax.set_title('Comparación de KPIs Promedios Jugador y Promedio Temporada 24/25', color='white')
ax.set_ylabel('Valor (0-9)', color='white')
ax.set_xticks(x + bar_width * (len(df_melted['label'].unique()) - 1) / 2)
ax.set_xticklabels(kpi_categories, color='white', rotation=45)

# Configurar leyenda
legend = ax.legend(
    title="Centrales Dvo.Alaves B",
    bbox_to_anchor=(1.05, 1),
    loc='upper left',
    facecolor=bg_color,
    framealpha=0,
    labelcolor='white',
    title_fontsize='medium'
)
legend.get_title().set_color('#00FFFF')

# Configurar estilo
ax.grid(True, linestyle='--', alpha=0.5, color='white')
ax.tick_params(colors='white')
for spine in ax.spines.values():
    spine.set_color(bg_color)

plt.tight_layout()
plt.show()


*dfs_Performance_per match, team, season, player & position*


In [ ]:
season_filter = '24/25'  # Temporada a filtrar:['21/22', '22/23', '23/24', '24/25']
team_filter = 'Deportivo Alavés B'  # Nombre del equipo a filtrar ['Deportivo Alavés Cad. A', 'Deportivo Alavés Juv. A', 'Deportivo Alavés C', 'Deportivo Alavés B' ]
position_filter = 'CB'  # Posición a filtrar: ['GK', 'CF', 'CM', 'RW', 'LB', 'RB', 'CB', 'CAM', 'LW', 'CDM', 'RM', 'LM', 'LWB', 'RWB', 'LF', 'RF']
player_filter = 'Alvaro Garcia Blanes'



filtered_df_Academia_matches = df_Academia_alaves_matches.loc[
    (df_Academia_alaves_matches['season_name'] == season_filter) &
    (df_Academia_alaves_matches['team_name'].str.contains(team_filter, case=False)) &
    (df_Academia_alaves_matches['main_position'] == position_filter) &
    (df_Academia_alaves_matches['player'] == player_filter)
]


# Crear una nueva columna temporal extrayendo el número de 'round_name'
filtered_df_Academia_matches['round_number'] = filtered_df_Academia_matches['round_name'].str.extract('(\d+)').fillna(0).astype(int)

# Ordenar el dataframe por 'season_name' y luego por 'round_number'
filtered_df_Academia_matches = filtered_df_Academia_matches.sort_values(by=['season_name', 'round_number'], ascending=[True, True])

# Eliminar la columna temporal 'round_number'
filtered_df_Academia_matches = filtered_df_Academia_matches.drop(columns=['round_number'])

filtered_df_Academia_matches.head(20)


*dfs_Performance_positions (Performance Ranking AcAdemia average SEASONS played)*


In [ ]:
# Definir el equipo, la posición y las temporadas deseadas
# Definir el equipo y las temporadas deseadas, incluyendo todas las posiciones posibles
team = 'Deportivo Alavés'
positions = ['GK', 'CF', 'CM', 'RW', 'LB', 'RB', 'CB', 'CAM', 'LW', 'CDM', 'RM', 'LM', 'LWB', 'RWB', 'LF', 'RF']
seasons = ['21/22', '22/23', '23/24', '24/25']



# Filtrar el DataFrame según los parámetros definidos
df_players_KPIranking = df_KPI_players_position[
    (df_KPI_players_position['team_name'].str.startswith(team)) &
    (df_KPI_players_position['main_position'].isin(positions)) &
    (df_KPI_players_position['season_name'].isin(seasons)) 
]


# Ordenar el DataFrame por jugador y temporada en orden ascendente para obtener el primer equipo
df_players_KPIranking = df_players_KPIranking.sort_values(by=['player', 'season_name'])

# Obtener el equipo de la primera temporada jugada para cada jugador
df_players_KPIranking['first_team'] = df_players_KPIranking.groupby('player')['team_name'].transform('first')

# Luego ordenar el DataFrame por jugador y temporada de forma descendente para obtener el último equipo
df_players_KPIranking = df_players_KPIranking.sort_values(by=['player', 'season_name'], ascending=[True, False])

# Obtener el equipo de la última temporada jugada para cada jugador
df_players_KPIranking['current_team'] = df_players_KPIranking.groupby('player')['team_name'].transform('first')

# Obtener la lista de temporadas jugadas por cada jugador
df_players_KPIranking['seasons_played'] = df_players_KPIranking.groupby('player')['season_name'].transform(lambda x: ', '.join(sorted(x.unique())))

# Calcular la cantidad de temporadas únicas en las que cada jugador ha participado
df_players_KPIranking['season_count'] = df_players_KPIranking.groupby('player')['season_name'].transform('nunique')

# Agrupar por jugador único y calcular el promedio de las métricas relacionadas a los partidos
df_players_KPIranking = df_players_KPIranking.groupby('player').agg({
    'back_number': 'first',         # Número de camiseta
    'player_id': 'first',            # ID del jugador
    'main_position': 'first',        # Posición principal
    'first_team': 'first',           # Primer equipo de la primera temporada
    'current_team': 'first',         # Último equipo en la última temporada
    'season_count': 'first',         # Cantidad de temporadas jugadas
    'seasons_played': 'first',       # Lista de temporadas jugadas
    'match': 'count', 

               
    # Promedio de métricas relacionadas al rendimiento
    'play_time_minutes': 'mean',
    'KPI_condicional': 'mean',
    'KPI_construccion_ataque': 'mean',
    'KPI_progresion': 'mean',
    'KPI_habilidad_individual': 'mean',
    'KPI_peligro_generado': 'mean',
    'KPI_finalizacion': 'mean',
    'KPI_eficacia_defensiva': 'mean',
    'KPI_juego_aereo': 'mean',
    'KPI_capacidad_recuperacion': 'mean',
    'KPI_posicionamiento_tactico': 'mean',
    'KPI_rendimiento': 'mean'
}).reset_index()


# Mostrar el resultado filtrado
df_players_KPIranking.sort_values(by='KPI_rendimiento', ascending=True)
df_players_KPIranking.reset_index(drop=True, inplace=True)
df_players_KPIranking.index += 1

df_players_KPIranking.head(25).round(2)

#  Dataframe average de temporadas para visualizaciones

In [ ]:

# Definir el equipo y las temporadas deseadas, incluyendo todas las posiciones posibles
team = 'Deportivo Alavés'
positions = ['GK', 'CF', 'CM', 'RW', 'LB', 'RB', 'CB', 'CAM', 'LW', 'CDM', 'RM', 'LM', 'LWB', 'RWB', 'LF', 'RF']
seasons = ['21/22', '22/23', '23/24', '24/25']

# Filtrar el DataFrame según los parámetros definidos, sin importar la posición
df_players_KPIranking = df_KPI_players_position[
    (df_KPI_players_position['team_name'].str.startswith(team)) &
    (df_KPI_players_position['main_position'].isin(positions)) &
    (df_KPI_players_position['season_name'].isin(seasons))
]


# Agregar columnas para la primera temporada y equipo, y la última temporada y equipo
df_players_KPIranking['seasons_played'] = df_players_KPIranking.groupby('player')['season_name'].transform(lambda x: ', '.join(sorted(x.unique())))
df_players_KPIranking['season_count'] = df_players_KPIranking.groupby('player')['season_name'].transform('nunique')

df_players_KPIranking['first_team'] = df_players_KPIranking.sort_values('season_name').groupby('player')['team_name'].transform('first')
df_players_KPIranking['current_team'] = df_players_KPIranking.sort_values('season_name').groupby('player')['team_name'].transform('last')

# Agrupar por jugador único y calcular el promedio de las métricas relacionadas a los partidos
df_players_KPIranking = df_players_KPIranking.groupby('player').agg({
    'back_number': 'first',        
    'player_id': 'first',            
    'main_position': 'first',                
    'first_team': 'first',   
    'current_team': 'first',         
    'season_count': 'first',         
    'seasons_played': 'first',       
    'match': 'count',                

    # Promedio de métricas relacionadas al rendimiento
    'play_time_minutes': 'mean',
    'KPI_condicional': 'mean',
    'KPI_construccion_ataque': 'mean',
    'KPI_progresion': 'mean',
    'KPI_habilidad_individual': 'mean',
    'KPI_peligro_generado': 'mean',
    'KPI_finalizacion': 'mean',
    'KPI_eficacia_defensiva': 'mean',
    'KPI_juego_aereo': 'mean',
    'KPI_capacidad_recuperacion': 'mean',
    'KPI_posicionamiento_tactico': 'mean',
    'KPI_rendimiento': 'mean'
}).reset_index()

# Columnas de KPI para normalizar
kpi_columns = [
    'KPI_condicional', 'KPI_construccion_ataque', 'KPI_progresion', 
    'KPI_habilidad_individual', 'KPI_peligro_generado', 'KPI_finalizacion', 
    'KPI_eficacia_defensiva', 'KPI_juego_aereo', 'KPI_capacidad_recuperacion', 
    'KPI_posicionamiento_tactico', 'KPI_rendimiento'
]



# Filtrar filas donde 'player' no contenga la palabra 'Unknown'
df_players_KPIranking = df_players_KPIranking[~df_players_KPIranking['player'].str.contains('Unknown', case=False, na=False)]

# Convertir la columna 'match' a numérica, remplazando valores no convertibles por NaN, luego eliminar estos registros
df_players_KPIranking['match'] = pd.to_numeric(df_players_KPIranking['match'], errors='coerce')
df_players_KPIranking = df_players_KPIranking.dropna(subset=['match'])

# Filtrar jugadores por PARTIDOS JUGADOS por temporada
#df_players_KPIranking = df_players_KPIranking[df_players_KPIranking['match'] >10]

# Función para normalizar entre 1 y 9
def normalizar(valores):
    min_val = valores.min()
    max_val = valores.max()
    # Evitar división por cero si todos los valores son iguales
    if min_val == max_val:
        return pd.Series([5] * len(valores))  # Asignar 5 si todos los valores son iguales
    else:
        return 1 + (valores - min_val) * (9 - 1) / (max_val - min_val)

# Aplicar normalización a cada columna de KPI
for kpi in kpi_columns:
    df_players_KPIranking[kpi] = normalizar(df_players_KPIranking[kpi]).round(2)

# Ordenar y mostrar resultados finales
df_players_KPIranking_normalizado = df_players_KPIranking.sort_values(by='KPI_rendimiento', ascending=False).reset_index(drop=True)
df_players_KPIranking_normalizado.index += 1  # Ajustar el índice para mostrar como ranking
df_players_KPIranking_normalizado[['player', 'back_number', 'player_id', 'main_position', 'first_team',
                                   'current_team', 'season_count', 'seasons_played', 
                                   'match'] + kpi_columns]



#Guardamos el DataFrame LATERALES IZDOS en archivo CSV
df_KPIplayers_Academia = df_players_KPIranking_normalizado
df_KPIplayers_Academia.to_csv('df_KPIplayers_Academia.csv', index=False)
df_KPIplayers_Academia.round(2).head(50)


*dfs_Performance_positions_NORMALIZED*


In [ ]:


# Definir el equipo, la posición y las temporadas deseadas
current_team = 'Deportivo Alavés'
#positions = ['GK', 'CF', 'CM', 'RW', 'LB', 'RB', 'CB', 'CAM', 'LW', 'CDM', 'RM', 'LM', 'LWB', 'RWB', 'LF', 'RF']
position = ['RB']
seasons = ['21/22', '22/23', '23/24', '24/25']
#seasons = ['24/25']
league = ['D. Honor Juvenil - G2', 'Tercera RFEF - G4', 'Segunda RFEF - G2','División de Honor - Araba', 'Copa Vasca Cadete', 'Liga Vasca Cadete']




# Filtrar el DataFrame según los parámetros definidos
df_players_KPIranking = df_KPI_players_position[
    (df_KPI_players_position['team_name'].str.startswith(current_team)) &
    (df_KPI_players_position['main_position'].isin(position)) &
    (df_KPI_players_position['season_name'].isin(seasons)) 
]


# Ordenar el DataFrame por jugador y temporada en orden ascendente para obtener el primer equipo
df_players_KPIranking = df_players_KPIranking.sort_values(by=['player', 'season_name'])

# Obtener el equipo de la primera temporada jugada para cada jugador
df_players_KPIranking['first_team'] = df_players_KPIranking.groupby('player')['team_name'].transform('first')

# Luego ordenar el DataFrame por jugador y temporada de forma descendente para obtener el último equipo
df_players_KPIranking = df_players_KPIranking.sort_values(by=['player', 'season_name'], ascending=[True, False])

# Obtener el equipo de la última temporada jugada para cada jugador
df_players_KPIranking['current_team'] = df_players_KPIranking.groupby('player')['team_name'].transform('first')

# Obtener la lista de temporadas jugadas por cada jugador
df_players_KPIranking['seasons_played'] = df_players_KPIranking.groupby('player')['season_name'].transform(lambda x: ', '.join(sorted(x.unique())))

# Calcular la cantidad de temporadas únicas en las que cada jugador ha participado
df_players_KPIranking['season_count'] = df_players_KPIranking.groupby('player')['season_name'].transform('nunique')

# Agrupar por jugador único y calcular el promedio de las métricas relacionadas a los partidos
df_players_KPIranking = df_players_KPIranking.groupby('player').agg({
    'back_number': 'first',         # Número de camiseta
    'player_id': 'first',            # ID del jugador
    'main_position': 'first',        # Posición principal
    'first_team': 'first',           # Primer equipo de la primera temporada
    'current_team': 'first',         # Último equipo en la última temporada
    'season_count': 'first',         # Cantidad de temporadas jugadas
    'seasons_played': 'first',       # Lista de temporadas jugadas
    'match': 'count', 

               
    # Promedio de métricas relacionadas al rendimiento
    'play_time_minutes': 'mean',
    'KPI_condicional': 'mean',
    'KPI_construccion_ataque': 'mean',
    'KPI_progresion': 'mean',
    'KPI_habilidad_individual': 'mean',
    'KPI_peligro_generado': 'mean',
    'KPI_finalizacion': 'mean',
    'KPI_eficacia_defensiva': 'mean',
    'KPI_juego_aereo': 'mean',
    'KPI_capacidad_recuperacion': 'mean',
    'KPI_posicionamiento_tactico': 'mean',
    'KPI_rendimiento': 'mean'
}).reset_index()

# Columnas de KPI que queremos normalizar
kpi_columns = [
    'KPI_condicional', 'KPI_construccion_ataque', 'KPI_progresion', 
    'KPI_habilidad_individual', 'KPI_peligro_generado', 'KPI_finalizacion', 
    'KPI_eficacia_defensiva', 'KPI_juego_aereo', 'KPI_capacidad_recuperacion', 
    'KPI_posicionamiento_tactico', 'KPI_rendimiento'
]

# Convertir la columna 'match' a numérica, remplazando valores no convertibles por NaN, luego eliminar estos registros
df_players_KPIranking['match'] = pd.to_numeric(df_players_KPIranking['match'], errors='coerce')
df_players_KPIranking = df_players_KPIranking.dropna(subset=['match'])

# Filtrar jugadores que hayan jugado más de 5 partidos por temporada
df_players_KPIranking = df_players_KPIranking[df_players_KPIranking['match'] >1]


# Función para normalizar entre 1 y 9
def normalizar(valores):
    min_val = valores.min()
    max_val = valores.max()
    # Evitar división por cero en caso de que todos los valores sean iguales
    if min_val == max_val:
        return pd.Series([5] * len(valores))  # Si todos los valores son iguales, asignar 5 (punto medio de 1 a 9)
    else:
        return 1 + (valores - min_val) * (8.9 - 1.94) / (max_val - min_val) # Valores max y min para centrales
        #return 1 + (valores - min_val) * (9 - 1.71) / (max_val - min_val) # Valores max y min para laterales 
        #return 1 + (valores - min_val) * (7 - 1.8) / (max_val - min_val) # Valores max y min para Mediocentros defensivos
        #return 1 + (valores - min_val) * (8.23 - 1.8) / (max_val - min_val) # Valores max y min para Mediocentros 
        #return 1 + (valores - min_val) * (9 - 1.8) / (max_val - min_val) # Valores max y min para Mediocentros ofensivos
        #return 1 + (valores - min_val) * (8.9 - 1.94) / (max_val - min_val) # Valores max y min para extremos y carrileros
        #return 1 + (valores - min_val) * (8.7 - 1.33) / (max_val - min_val) # Valores max y min para delanteros



# Aplicar normalización a cada columna de KPI
for kpi in kpi_columns:
    df_players_KPIranking[kpi] = normalizar(df_players_KPIranking[kpi]).round(2)

# Mostrar el resultado final, ordenado por KPI_rendimiento
df_players_KPIranking_normalizado = df_players_KPIranking.sort_values(by='KPI_rendimiento', ascending=False).round(2)

# Resetear el índice y añadir un índice de ranking
df_players_KPIranking_normalizado.reset_index(drop=True, inplace=True)
df_players_KPIranking_normalizado.index += 1

# Mostrar los primeros 25 resultados, incluyendo el primer equipo, último equipo y edad
df_players_KPIranking_normalizado[['player', 'back_number', 'player_id', 'main_position', 
                                   'first_team', 'current_team', 'season_count', 'seasons_played', 
                                   'match'] + kpi_columns].head(50)


# Definir todas las posiciones posibles:['GK', 'CF', 'CM', 'RW', 'LB', 'RB', 'CB', 'CAM', 'LW', 'CDM', 'RM', 'LM', 'LWB', 'RWB', 'LF', 'RF']

#Guardamos el DataFrame CENTRALES en archivo CSV
#df_KPIs_Centrales_Academia = df_players_KPIranking_normalizado
#df_KPIs_Centrales_Academia.to_csv('df_KPIs_Centrales_Academia.csv', index=False)
#df_KPIs_Centrales_Academia

#Guardamos el DataFrame LATERALES IZDOS en archivo CSV
#df_KPIs_Lateralesizdos_Academia = df_players_KPIranking_normalizado
#df_KPIs_Lateralesizdos_Academia.to_csv('df_KPIs_lateralesizdos_Academia.csv', index=False)
#df_KPIs_Lateralesizdos_Academia

#Guardamos el DataFrame LATERALES DCHOS en archivo CSV
df_KPIs_Lateralesdchos_Academia = df_players_KPIranking_normalizado
df_KPIs_Lateralesdchos_Academia.to_csv('df_KPIs_Lateralesdchos_Academia.csv', index=False)
df_KPIs_Lateralesdchos_Academia

#Guardamos el DataFrame CARRILEROS IZDOS en archivo CSV
#df_KPIs_Carrilerosizdos_Academia = df_players_KPIranking_normalizado
#df_KPIs_Carrilerosizdos_Academia.to_csv('df_KPIs_Carrilerosizdos_Academia.csv', index=False)
#df_KPIs_Carrilerosizdos_Academia

#Guardamos el DataFrame CARRILEROS DCHOS en archivo CSV
#df_KPIs_Carrilerosdchos_Academia = df_players_KPIranking_normalizado
#df_KPIs_Carrilerosdchos_Academia.to_csv('df_KPIs_Carrilerosdchos_Academia.csv', index=False)
#df_KPIs_Carrilerosdchos_Academia
 
#Guardamos el DataFrame MEDIOCENTROS en archivo CSV
#df_KPIs_Mediocentros_Academia = df_players_KPIranking_normalizado
#df_KPIs_Mediocentros_Academia.to_csv('df_KPIs_Mediocentros_Academia.csv', index=False)
#df_KPIs_Mediocentros_Academia

#Guardamos el DataFrame MEDIOS DEFENSIVOS en archivo CSV
#df_KPIs_McDefensivos_Academia = df_players_KPIranking_normalizado
#df_KPIs_McDefensivos_Academia.to_csv('df_KPIs_McDefensivos_Academia.csv', index=False)
#df_KPIs_McDefensivos_Academia

#Guardamos el DataFrame MEDIOS OEFENSIVOS en archivo CSV
#df_KPIs_McOfensivos_Academia = df_players_KPIranking_normalizado
#df_KPIs_McOfensivos_Academia.to_csv('df_KPIs_McOfensivos.csv', index=False)
#df_KPIs_McOfensivos_Academia

#Guardamos el DataFrame EXTREMOS DCHOS en archivo CSV
#df_KPIs_Extremosdchos_Academia = df_players_KPIranking_normalizado
#df_KPIs_Extremosdchos_Academia.to_csv('df_KPIs_Extremosdchos_Academia.csv', index=False)
#df_KPIs_Extremosdchos_Academia

#Guardamos el DataFrame EXTREMOS IZDOS en archivo CSV
#df_KPIs_Extremosizdos_Academia = df_players_KPIranking_normalizado
#df_KPIs_Extremosizdos_Academia.to_csv('df_KPIs_Extremosizdos_Academia.csv', index=False)
#df_KPIs_Extremosizdos_Academia

#Guardamos el DataFrame DELANTEROS en archivo CSV
#df_KPIs_Delanteros_Academia = df_players_KPIranking_normalizado
#df_KPIs_Delanteros_Academia.to_csv('df_KPIs_Delanteros_Academia.csv', index=False)
#df_KPIs_Delanteros_Academia





## Visualizacion rendimiento por posicion y temporada en la Academia

In [ ]:


#position = ['GK', 'CF', 'CM', 'RW', 'LB', 'RB', 'CB', 'CAM', 'LW', 'CDM', 'RM', 'LM', 'LWB', 'RWB', 'LF', 'RF']
#df_position = df_KPIs_Centrales_Academia
#df_position = df_KPIs_lateralesizdos_Academia
df_position = df_KPIs_Lateralesdchos_Academia
#df_position = df_KPIs_Carrilerosizdos_Academia
#df_position = df_KPIs_Carrilerosdchos_Academia
#df_position = df_KPIs_Mediocentros_Academia
#df_position = df_KPIs_McDefensivos_Academia
#df_position = df_KPIs_McOfensivos_Academia
#df_position = df_KPIs_Extremosdchos_Academia
#df_position = df_KPIs_Extremosizdos_Academia
#df_position = df_KPIs_Delanteros_Academia

# Crear DataFrame
df = pd.DataFrame(df_position)

# Asignar colores por equipo
current_team_colors = {
    'Deportivo Alavés Cad. A': 'blue',
    'Deportivo Alavés C': 'pink',
    'Deportivo Alavés B': 'red',
    'Deportivo Alavés Juv. A': 'yellow'
}
df['color'] = df['current_team'].map(current_team_colors)
df['color'] = df['color'].fillna('grey')  # Color por defecto si el equipo no está en el diccionario

# Crear una nueva columna para el texto del eje Y: Nombre + número de partidos
df['player_label'] = df['player'] + ' (' + df['match'].astype(str) + ' matches)'

# Ordenar los datos de menor a mayor rendimiento
df = df.sort_values(by='KPI_rendimiento', ascending=True)

# Color de fondo para la figura y los ejes
bg_color = '#0E3F5C'

# Configurar la figura y el fondo de los ejes
fig, ax = plt.subplots(figsize=(12, 8), facecolor=bg_color)
ax.set_facecolor(bg_color)

# Graficar en barras horizontales
bars = ax.barh(df['player_label'], df['KPI_rendimiento'], color=df['color'], height=0.5)
ax.set_xlabel('KPI_rendimiento', color='white')  # Color de la etiqueta del eje X
ax.set_title('Performance Index Centrales Academia (Temporadas:21/22 a 24/25)', color='white')

# Cambiar el color de los nombres de los jugadores a blanco
ax.tick_params(axis='y', colors='white', labelsize=10)  # Color y tamaño de las etiquetas del eje Y

# Eliminar marcas y etiquetas del eje X
ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)

# Añadir los valores del índice sobre cada barra
for bar in bars:
    xval = bar.get_width()
    ax.text(xval, bar.get_y() + bar.get_height() / 2.0, round(xval, 3), va='center', ha='left', size=10, color='white')

# Ocultar los bordes del gráfico
for spine in ax.spines.values():
    spine.set_visible(False)

# Añadir leyenda
handles = [plt.Line2D([0], [0], color=color, lw=4, label=current_team) for current_team, color in current_team_colors.items()]
legend = ax.legend(handles=handles, title="Equipos Academia", facecolor=bg_color, framealpha=0)

# Configurar el color del título y las etiquetas de la leyenda
legend.get_title().set_color("white")
for text in legend.get_texts():
    text.set_color("white")

# Mostrar el gráfico
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Leer CSVs y guardar como Parquet
df_KPIs_Centrales_Academia.to_parquet('../academyApp/data/archivos_parquet/df_KPIs_Centrales_Academia.parquet', index=False)
df_KPIs_Lateralesizdos_Academia.to_parquet('../academyApp/data/archivos_parquet/df_KPIs_Lateralesizdos_Academia.parquet', index=False)
df_KPIs_Lateralesdchos_Academia.to_parquet('../academyApp/data/archivos_parquet/df_KPIs_Lateralesdchos_Academia.parquet', index=False)
df_KPIs_Mediocentros_Academia.to_parquet('../academyApp/data/archivos_parquet/df_KPIs_Mediocentros_Academia.parquet', index=False)
df_KPIs_McDefensivos_Academia.to_parquet('../academyApp/data/archivos_parquet/df_KPIs_McDefensivos_Academia.parquet', index=False)
df_KPIs_McOfensivos_Academia.to_parquet('../academyApp/data/archivos_parquet/df_KPIs_McOfensivos_Academia.parquet', index=False)
df_KPIs_Extremosdchos_Academia.to_parquet('../academyApp/data/archivos_parquet/df_KPIs_Extremosdchos_Academia.parquet', index=False)
df_KPIs_Extremosizdos_Academia.to_parquet('../academyApp/data/archivos_parquet/df_KPIs_Extremosizdos_Academia.parquet', index=False)
df_KPIs_Delanteros_Academia.to_parquet('../academyApp/data/archivos_parquet/df_KPIs_Delanteros_Academia.parquet', index=False)


# Diccionario que mapea posiciones a DataFrames
position_to_dataframe = {
    'CB': df_KPIs_Centrales_Academia,
    'LB': df_KPIs_Lateralesizdos_Academia,
    'RB': df_KPIs_Lateralesdchos_Academia,
    'CM': df_KPIs_Mediocentros_Academia,
    'RM': df_KPIs_Mediocentros_Academia,
    'LM': df_KPIs_Mediocentros_Academia,
    'CDM': df_KPIs_McDefensivos_Academia,
    'CAM': df_KPIs_McOfensivos_Academia,
    'RW': df_KPIs_Extremosdchos_Academia,
    'LW': df_KPIs_Extremosizdos_Academia,
    'CF': df_KPIs_Delanteros_Academia,
    'RF': df_KPIs_Delanteros_Academia,
    'LF': df_KPIs_Delanteros_Academia
}

# Asignar colores por equipo
current_team_colors = {
    'Deportivo Alavés Cad. A': 'blue',
    'Deportivo Alavés C': 'pink',
    'Deportivo Alavés B': 'red',
    'Deportivo Alavés Juv. A': 'yellow'
}

# Selector para elegir la posición de jugador
position_selector = widgets.Dropdown(
    options=position_to_dataframe.keys(),
    value='CB',  # Valor predeterminado
    description='Posición:'
)

# Función para actualizar el gráfico basado en la selección de posición
def update_chart(selected_position):
    # Obtener el DataFrame correspondiente a la posición seleccionada
    df_position = position_to_dataframe[selected_position]
    
    # Filtrar DataFrame para que coincida solo con la posición elegida
    df_filtered = df_position[df_position['main_position'] == selected_position]

    # Crear un nuevo DataFrame para graficar
    df = pd.DataFrame(df_filtered)

    # Asignar color basado en el equipo
    df['color'] = df['current_team'].map(current_team_colors).fillna('grey')  # Color gris si el equipo no está en el diccionario

    # Crear la columna para el eje Y: Nombre + número de partidos
    df['player_label'] = df['player'] + ' (' + df['match'].astype(str) + ' matches)'

    # Ordenar datos por rendimiento
    df = df.sort_values(by='KPI_rendimiento', ascending=True)

    # Configuración de la figura y el fondo
    bg_color = '#0E3F5C'
    fig, ax = plt.subplots(figsize=(12, 8), facecolor=bg_color)
    ax.set_facecolor(bg_color)

    # Graficar barras horizontales
    bars = ax.barh(df['player_label'], df['KPI_rendimiento'], color=df['color'], height=0.5)
    ax.set_xlabel('KPI Rendimiento', color='white')
    ax.set_title(f'Performance Index {selected_position} Academia (Temporadas:21/22 a 24/25)', color='white')

    # Personalizar etiquetas
    ax.tick_params(axis='y', colors='white', labelsize=10)
    ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)

    # Mostrar los valores de índice en cada barra
    for bar in bars:
        xval = bar.get_width()
        ax.text(xval, bar.get_y() + bar.get_height() / 2.0, round(xval, 3), va='center', ha='left', size=10, color='white')

    # Ocultar los bordes del gráfico
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Añadir leyenda
    handles = [plt.Line2D([0], [0], color=color, lw=4, label=current_team) for current_team, color in current_team_colors.items()]
    legend = ax.legend(handles=handles, title="Equipos Academia", facecolor=bg_color, framealpha=0)

    # Configurar colores del título y texto de la leyenda
    legend.get_title().set_color("white")
    for text in legend.get_texts():
        text.set_color("white")

    # Mostrar el gráfico
    plt.show()

# Crear el interactivo para actualizar el gráfico al cambiar la posición
display(position_selector)
widgets.interactive(update_chart, selected_position=position_selector)


In [ ]:

# Asumimos que el DataFrame df_KPIplayers_Academia ya está cargado
# y contiene las columnas necesarias.

# Definir el color de fondo
bg_color = '#0E3F5C'

# Asumir que ya tienes los datos cargados en df_KPIplayers_Academia
# y que las columnas relevantes son 'main_position', 'current_team', 'seasons_played', 'KPI_rendimiento', etc.


# Parametros de selección:
#team = ['all_teams, 'Deportivo Alavés B', 'Deportivo Alavés C', 'Deportivo Alavés Juv. A', Deportivo Alavés Cad. A']
#positions = ['all_positions', GK', 'CF', 'CM', 'RW', 'LB', 'RB', 'CB', 'CAM', 'LW', 'CDM', 'RM', 'LM', 'LWB', 'RWB', 'LF', 'RF']
#seasons = ['21/22', '22/23', '23/24', '24/25']
#yaxis KPI = ['KPI_condicional', 'KPI_construccion_ataque', 'KPI_progresion', 'KPI_habilidad_individual', 'KPI_peligro_generado', 'KPI_finalizacion', 
# 'KPI_eficacia_defensiva', 'KPI_juego_aereo', 'KPI_capacidad_recuperacion', 'KPI_posicionamiento_tactico', 'KPI_rendimiento']


# Selección manual de parámetros:
selected_kpi = 'KPI_condicional'  # Reemplaza con el KPI que quieras usar
selected_position = 'CB'  # Reemplaza con la posición deseada (por ejemplo, 'CB', 'LB', 'RW', etc.)
selected_team = 'Deportivo Alavés B' # Selecciona el equipo deseado
season_24_25 = False  # Si es solo para la temporada 24/25, ponlo en True
rendimiento_range = [0, 9]  # Rango de KPI_rendimiento, ajusta según sea necesario


# Filtrar el DataFrame manualmente
df_filtered = df_KPIplayers_Academia.copy()

# Filtrar por posición si no es 'all_positions'
if selected_position != 'all_positions':
    df_filtered = df_filtered[df_filtered['main_position'] == selected_position]

# Filtrar por equipo si no es 'all_teams'
if selected_team != 'all_teams':
    df_filtered = df_filtered[df_filtered['current_team'] == selected_team]

# Filtrar por temporada '24/25' si el checkbox está activado
if season_24_25:
    df_filtered = df_filtered[df_filtered['seasons_played'].str.contains('24/25')]

# Filtrar por rango de KPI_rendimiento
rendimiento_min, rendimiento_max = rendimiento_range
df_filtered = df_filtered[(df_filtered['KPI_rendimiento'] >= rendimiento_min) & (df_filtered['KPI_rendimiento'] <= rendimiento_max)]

# Crear el gráfico de dispersión con fondo personalizado
fig, ax = plt.subplots(figsize=(15, 10), facecolor=bg_color)
ax.set_facecolor(bg_color)

# Asignación de colores según equipo
team_colors = {
    'Deportivo Alavés Cad. A': 'blue',
    'Deportivo Alavés C': 'pink',
    'Deportivo Alavés B': 'red',
    'Deportivo Alavés Juv. A': 'yellow'
}

# Crear scatter plot con color según equipo y borde en negro
for team, color in team_colors.items():
    team_data = df_filtered[df_filtered['current_team'] == team]
    ax.scatter(
        team_data['KPI_rendimiento'],
        team_data[selected_kpi],
        color=color,
        edgecolor='white',
        label=team
    )

# Etiquetas de los jugadores con el formato solicitado
for i, row in df_filtered.iterrows():
    # Nombre del jugador y posición principal entre paréntesis
    player_label = f"{row['player']} ({row['main_position']})"
    # Número de temporadas solamente
    season_label = f"({int(row['season_count'])} seasons)"

    # Añadir las etiquetas en el gráfico con nombre a la derecha y detalles debajo
    ax.annotate(
        player_label, 
        (row['KPI_rendimiento'], row[selected_kpi]), 
        textcoords="offset points", 
        xytext=(5, 0),  # Posiciona el nombre del jugador a la derecha del punto
        ha='left', 
        color='white', 
        fontsize=8, 
        weight='normal'
    )
    # Etiqueta más pequeña debajo del nombre del jugador
    ax.annotate(
        season_label, 
        (row['KPI_rendimiento'], row[selected_kpi]), 
        textcoords="offset points", 
        xytext=(5, -10),  # Posiciona la información de las temporadas debajo del nombre del jugador
        ha='left', 
        color='white', 
        fontsize=6
    )

# Personalizar ejes y título del gráfico
ax.set_xlabel('KPI Rendimiento', color='white')
ax.set_ylabel(selected_kpi, color='white')
ax.set_title('Comparación de KPI Rendimiento vs KPI Seleccionado', color='white')

# Posicionar la leyenda fuera del gráfico, con fondo transparente y texto en blanco
legend = ax.legend(
    loc='center left',           # Posición en el margen derecho
    bbox_to_anchor=(1, 0.5),     # Ajustar la posición de la leyenda fuera del gráfico
    fontsize='small',             # Tamaño de fuente más pequeño
    frameon=False,                # Fondo transparente
    labelcolor='white'            # Texto de la leyenda en blanco
)

# Personalización del diseño de la gráfica
ax.grid(True, linestyle='--', alpha=0.5, color='white')
ax.tick_params(colors='white')
ax.spines['top'].set_color(bg_color)
ax.spines['right'].set_color(bg_color)
ax.spines['left'].set_color(bg_color)
ax.spines['bottom'].set_color(bg_color)

plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, HBox, VBox

# Define el color de fondo
bg_color = '#0E3F5C'

# Lista de opciones para los KPIs basados en las columnas del DataFrame
kpi_columns = [col for col in df_KPIplayers_Academia.columns if col.startswith('KPI_')]

# Crear widgets para los filtros de selección
kpi_selector = widgets.Dropdown(
    options=kpi_columns,
    value='KPI_condicional',  # Valor predeterminado
    description='KPI eje Y:'
)

position_selector = widgets.Dropdown(
    options=['all_positions'] + list(df_KPIplayers_Academia['main_position'].unique()),
    value='all_positions',  # Valor predeterminado
    description='Posición:'
)

team_selector = widgets.Dropdown(
    options=['all_teams'] + list(df_KPIplayers_Academia['current_team'].unique()),
    value='all_teams',  # Valor predeterminado
    description='Equipo:'
)

# Filtro de temporada: selecciona solo los jugadores que jugaron en la temporada "24/25"
season_selector = widgets.Checkbox(
    value=False,
    description="Solo temporada 24/25"
)

# Slider para limitar el rango de KPI_rendimiento
rendimiento_slider = widgets.FloatRangeSlider(
    value=[0, 9],
    min=0,
    max=9,
    step=0.1,
    description='KPI Rendimiento:',
    continuous_update=False
)

# Agrupar los widgets de filtro en una sola fila horizontal
filters = HBox([kpi_selector, position_selector, team_selector, season_selector, rendimiento_slider])

# Asignación de colores según equipo
team_colors = {
    'Deportivo Alavés Cad. A': 'blue',
    'Deportivo Alavés C': 'pink',
    'Deportivo Alavés B': 'red',
    'Deportivo Alavés Juv. A': 'yellow'
}

# Función para actualizar el gráfico basado en la selección del usuario
def update_scatter(selected_kpi, selected_position, selected_team, season_24_25, rendimiento_range):
    # Filtrar el DataFrame por posición, equipo y temporada
    df_filtered = df_KPIplayers_Academia.copy()

    # Filtrar por posición si no es "all_positions"
    if selected_position != 'all_positions':
        df_filtered = df_filtered[df_filtered['main_position'] == selected_position]

    # Filtrar por equipo si no es "all_teams"
    if selected_team != 'all_teams':
        df_filtered = df_filtered[df_filtered['current_team'] == selected_team]

    # Filtrar por temporada "24/25" si el checkbox está activado
    if season_24_25:
        df_filtered = df_filtered[df_filtered['seasons_played'].str.contains('24/25')]

    # Filtrar por rango de KPI_rendimiento
    rendimiento_min, rendimiento_max = rendimiento_range
    df_filtered = df_filtered[(df_filtered['KPI_rendimiento'] >= rendimiento_min) & (df_filtered['KPI_rendimiento'] <= rendimiento_max)]

    # Crear el gráfico de dispersión con fondo personalizado
    fig, ax = plt.subplots(figsize=(15, 10), facecolor=bg_color)
    ax.set_facecolor(bg_color)

    # Crear scatter plot con color según equipo y borde en negro
    for team, color in team_colors.items():
        team_data = df_filtered[df_filtered['current_team'] == team]
        ax.scatter(
            team_data['KPI_rendimiento'],
            team_data[selected_kpi],
            color=color,
            edgecolor='white',
            label=team
        )

    # Etiquetas de los jugadores con el formato solicitado
    for i, row in df_filtered.iterrows():
        # Nombre del jugador y posición principal entre paréntesis
        player_label = f"{row['player']} ({row['main_position']})"
        # Número de temporadas solamente
        season_label = f"({int(row['season_count'])} seasons)"

        # Añadir las etiquetas en el gráfico con nombre a la derecha y detalles debajo
        ax.annotate(
            player_label, 
            (row['KPI_rendimiento'], row[selected_kpi]), 
            textcoords="offset points", 
            xytext=(5, 0),  # Posiciona el nombre del jugador a la derecha del punto
            ha='left', 
            color='white', 
            fontsize=8, 
            weight='normal'
        )
        # Etiqueta más pequeña debajo del nombre del jugador
        ax.annotate(
            season_label, 
            (row['KPI_rendimiento'], row[selected_kpi]), 
            textcoords="offset points", 
            xytext=(5, -10),  # Posiciona la información de las temporadas debajo del nombre del jugador
            ha='left', 
            color='white', 
            fontsize=6
        )

    # Personalizar ejes y título del gráfico
    ax.set_xlabel('KPI Rendimiento', color='white')
    ax.set_ylabel(selected_kpi, color='white')
    ax.set_title('Comparación de KPI Rendimiento vs KPI Seleccionado', color='white')

    # Posicionar la leyenda fuera del gráfico, con fondo transparente y texto en blanco
    legend = ax.legend(
        loc='center left',           # Posición en el margen derecho
        bbox_to_anchor=(1, 0.5),     # Ajustar la posición de la leyenda fuera del gráfico
        fontsize='small',             # Tamaño de fuente más pequeño
        frameon=False,                # Fondo transparente
        labelcolor='white'            # Texto de la leyenda en blanco
    )

    # Personalización del diseño de la gráfica
    ax.grid(True, linestyle='--', alpha=0.5, color='white')
    ax.tick_params(colors='white')
    ax.spines['top'].set_color(bg_color)
    ax.spines['right'].set_color(bg_color)
    ax.spines['left'].set_color(bg_color)
    ax.spines['bottom'].set_color(bg_color)

    plt.show()

# Crear un interactivo para seleccionar el KPI en eje Y y aplicar filtros adicionales
ui = VBox([filters])  # Organizar los filtros en una caja horizontal
interact(
    update_scatter, 
    selected_kpi=kpi_selector, 
    selected_position=position_selector,
    selected_team=team_selector,
    season_24_25=season_selector,
    rendimiento_range=rendimiento_slider
)


#  Dataframes para visualizaciones

In [ ]:
import pandas as pd
df_KPIplayers_Academia= pd.read_csv('/Users/sevi/Documents/DATA ALAVES/df_KPIplayers_Academia.csv')
pd.set_option('display.max_columns', None)
df_KPIplayers_Academia.round(2).head(50)

In [ ]:
df_KPI_playersavg_normalizado.round(2)

In [ ]:
import pandas as pd

# Definir el equipo, la posición y las temporadas deseadas

#current_team = []'Deportivo Alavés B', 'Deportivo Alavés C', 'Deportivo Alavés Juv. A', 'Deportivo Alavés Cad. A'
current_team = 'Deportivo Alavés B'
#positions = ['GK', 'CF', 'CM', 'RW', 'LB', 'RB', 'CB', 'CAM', 'LW', 'CDM', 'RM', 'LM', 'LWB', 'RWB', 'LF', 'RF']
position = ['CB']
#seasons = ['21/22', '22/23', '23/24', '24/25']
seasons = ['24/25']
league = ['D. Honor Juvenil - G2', 'Tercera RFEF - G4', 'Segunda RFEF - G2','División de Honor - Araba', 'Copa Vasca Cadete', 'Liga Vasca Cadete']





# Filtrar el DataFrame según los parámetros definidos
df_players_KPIranking = df_KPI_players_position[
    (df_KPI_players_position['team_name'].str.startswith(current_team)) &
    (df_KPI_players_position['main_position'].isin(position)) &
    (df_KPI_players_position['season_name'] == season)
]


# Ordenar el DataFrame por jugador y temporada en orden ascendente para obtener el primer equipo
df_players_KPIranking = df_players_KPIranking.sort_values(by=['player', 'season_name'])

# Obtener el equipo de la primera temporada jugada para cada jugador
df_players_KPIranking['first_team'] = df_players_KPIranking.groupby('player')['team_name'].transform('first')

# Luego ordenar el DataFrame por jugador y temporada de forma descendente para obtener el último equipo
df_players_KPIranking = df_players_KPIranking.sort_values(by=['player', 'season_name'], ascending=[True, False])

# Obtener el equipo de la última temporada jugada para cada jugador
df_players_KPIranking['current_team'] = df_players_KPIranking.groupby('player')['team_name'].transform('first')

# Obtener la lista de temporadas jugadas por cada jugador
df_players_KPIranking['seasons_played'] = df_players_KPIranking.groupby('player')['season_name'].transform(lambda x: ', '.join(sorted(x.unique())))

# Calcular la cantidad de temporadas únicas en las que cada jugador ha participado
df_players_KPIranking['season_count'] = df_players_KPIranking.groupby('player')['season_name'].transform('nunique')

# Agrupar por jugador único y calcular el promedio de las métricas relacionadas a los partidos
df_players_KPIranking = df_players_KPIranking.groupby('player').agg({
    'back_number': 'first',         # Número de camiseta
    'player_id': 'first',            # ID del jugador
    'main_position': 'first',        # Posición principal
    'first_team': 'first',           # Primer equipo de la primera temporada
    'current_team': 'first',         # Último equipo en la última temporada
    'season_count': 'first',         # Cantidad de temporadas jugadas
    'seasons_played': 'first',       # Lista de temporadas jugadas
    'match': 'count', 

               
    # Promedio de métricas relacionadas al rendimiento
    'play_time_minutes': 'mean',
    'KPI_condicional': 'mean',
    'KPI_construccion_ataque': 'mean',
    'KPI_progresion': 'mean',
    'KPI_habilidad_individual': 'mean',
    'KPI_peligro_generado': 'mean',
    'KPI_finalizacion': 'mean',
    'KPI_eficacia_defensiva': 'mean',
    'KPI_juego_aereo': 'mean',
    'KPI_capacidad_recuperacion': 'mean',
    'KPI_posicionamiento_tactico': 'mean',
    'KPI_rendimiento': 'mean'
}).reset_index()

# Columnas de KPI que queremos normalizar
kpi_columns = [
    'KPI_condicional', 'KPI_construccion_ataque', 'KPI_progresion', 
    'KPI_habilidad_individual', 'KPI_peligro_generado', 'KPI_finalizacion', 
    'KPI_eficacia_defensiva', 'KPI_juego_aereo', 'KPI_capacidad_recuperacion', 
    'KPI_posicionamiento_tactico', 'KPI_rendimiento'
]

# Convertir la columna 'match' a numérica, remplazando valores no convertibles por NaN, luego eliminar estos registros
df_players_KPIranking['match'] = pd.to_numeric(df_players_KPIranking['match'], errors='coerce')
df_players_KPIranking = df_players_KPIranking.dropna(subset=['match'])

# Filtrar jugadores que hayan jugado más de 5 partidos por temporada
df_players_KPIranking = df_players_KPIranking[df_players_KPIranking['match'] >1]


# Función para normalizar entre 1 y 9
def normalizar(valores):
    min_val = valores.min()
    max_val = valores.max()
    # Evitar división por cero en caso de que todos los valores sean iguales
    if min_val == max_val:
        return pd.Series([5] * len(valores))  # Si todos los valores son iguales, asignar 5 (punto medio de 1 a 9)
    else:
        return 1 + (valores - min_val) * (8.9 - 1.94) / (max_val - min_val) # Valores max y min para centrales
        #return 1 + (valores - min_val) * (9 - 1.71) / (max_val - min_val) # Valores max y min para laterales 
        #return 1 + (valores - min_val) * (7 - 1.8) / (max_val - min_val) # Valores max y min para Mediocentros defensivos
        #return 1 + (valores - min_val) * (8.23 - 1.8) / (max_val - min_val) # Valores max y min para Mediocentros 
        #return 1 + (valores - min_val) * (9 - 1.8) / (max_val - min_val) # Valores max y min para Mediocentros ofensivos
        #return 1 + (valores - min_val) * (8.9 - 1.94) / (max_val - min_val) # Valores max y min para extremos y carrileros
        #return 1 + (valores - min_val) * (8.7 - 1.33) / (max_val - min_val) # Valores max y min para delanteros



# Aplicar normalización a cada columna de KPI
for kpi in kpi_columns:
    df_players_KPIranking[kpi] = normalizar(df_players_KPIranking[kpi]).round(2)

# Mostrar el resultado final, ordenado por KPI_rendimiento
df_players_KPIranking_normalizado = df_players_KPIranking.sort_values(by='KPI_rendimiento', ascending=False).round(2)

# Resetear el índice y añadir un índice de ranking
df_players_KPIranking_normalizado.reset_index(drop=True, inplace=True)
df_players_KPIranking_normalizado.index += 1

# Mostrar los primeros 25 resultados, incluyendo el primer equipo, último equipo y edad
df_players_KPIranking_normalizado[['player', 'back_number', 'player_id', 'main_position', 
                                   'first_team', 'current_team', 'season_count', 'seasons_played', 
                                   'match'] + kpi_columns].head(50)


# Crear una fila promedio para los KPIs
fila_promedio = df_players_KPIranking_normalizado[kpi_columns].mean().round(2)

# Añadir columnas adicionales para la fila promedio
fila_promedio['player'] = 'Player.avg'
fila_promedio['back_number'] = 'Average'
fila_promedio['player_id'] = 'Average'
fila_promedio['main_position'] = ', '.join(position)  # Mostrar posiciones seleccionadas
fila_promedio['first_team'] = 'Average'
fila_promedio['current_team'] = current_team
fila_promedio['season_count'] = 1
fila_promedio['seasons_played'] = ', '.join(seasons)  # Mostrar temporadas seleccionadas
fila_promedio['play_time_minutes'] = df_players_KPIranking_normalizado['play_time_minutes'].mean().round(2)
fila_promedio['match'] = df_players_KPIranking_normalizado['match'].mean().round(2)  # Promedio de partidos

# Convertir la serie en un DataFrame de una fila y concatenar al DataFrame original
df_players_KPIranking_normalizado = pd.concat(
    [df_players_KPIranking_normalizado, fila_promedio.to_frame().T],
    ignore_index=True
)

# Mostrar el DataFrame con la fila promedio añadida
df_players_KPIranking_normalizado[['player', 'back_number', 'player_id', 'main_position', 
                                   'first_team', 'current_team', 'season_count', 'seasons_played', 
                                   'match'] + kpi_columns].head(50)



# DataFrame CENTRALES POR TEMPORADA
df_KPIs_Centrales_Academia = df_players_KPIranking_normalizado
df_KPIs_Centrales_Academia

# DataFrame LATERALES IZDOS POR TEMPORADA
#df_KPIs_Lateralesizdos_Academia = df_players_KPIranking_normalizado
#df_KPIs_Lateralesizdos_Academia

#DataFrame LATERALES DCHOS POR TEMPORADA
#df_KPIs_Lateralesdchos_Academia = df_players_KPIranking_normalizado
#df_KPIs_Lateralesdchos_Academia

# DataFrame CARRILEROS IZDOS POR TEMPORADA
#df_KPIs_Carrilerosizdos_Academia = df_players_KPIranking_normalizado
#df_KPIs_Carrilerosizdos_Academia

#GDataFrame CARRILEROS DCHOS POR TEMPORADA
#df_KPIs_Carrilerosdchos_Academia = df_players_KPIranking_normalizado
#df_KPIs_Carrilerosdchos_Academia
 
# DataFrame MEDIOCENTROS POR TEMPORADA
#df_KPIs_Mediocentros_Academia = df_players_KPIranking_normalizado

#df_KPIs_Mediocentros_Academia

#DataFrame MEDIOS DEFENSIVOS POR TEMPORADA
#df_KPIs_McDefensivos_Academia = df_players_KPIranking_normalizado

#df_KPIs_McDefensivos_Academia

#DataFrame MEDIOS OEFENSIVOS POR TEMPORADA
#df_KPIs_McOfensivos_Academia = df_players_KPIranking_normalizado
#df_KPIs_McOfensivos_Academia

#DataFrame EXTREMOS DCHOS POR TEMPORADA
#df_KPIs_Extremosdchos_Academia = df_players_KPIranking_normalizado
#df_KPIs_Extremosdchos_Academia

#DataFrame EXTREMOS IZDOS POR TEMPORADA
#df_KPIs_Extremosizdos_Academia = df_players_KPIranking_normalizado
#df_KPIs_Extremosizdos_Academia

#DataFrame DELANTEROS POR TEMPORADA
#df_KPIs_Delanteros_Academia = df_players_KPIranking_normalizado
#df_KPIs_Delanteros_Academia









## Visualizaciones Perfil de Rendimiento/temporada

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from math import pi

# Colores para los jugadores
col1 = '#1E90FF' # Color para el primer jugador
col2 =  '#D3D3D3'  # Color para el segundo jugador

# Función para crear el radar chart
def radarchart_KPIs(player1, player2, df):
    # Filtrar los datos del DataFrame para obtener los valores de los jugadores seleccionados
    player1_data = df[df['player'] == player1].iloc[0, 10:].values  # KPI columns start at index 10
    player2_data = df[df['player'] == player2].iloc[0, 10:].values

    # Seleccionar categorías (todas excepto las columnas iniciales de información del jugador)
    categories = df.columns[10:]  # Seleccionar solo las columnas de KPI
    n_categories = len(categories)

    # Cerrar el gráfico con el primer valor para cerrar el círculo
    values_player1 = player1_data.tolist() + player1_data[:1].tolist()
    values_player2 = player2_data.tolist() + player2_data[:1].tolist()

    # Ángulos para las categorías
    angles = [n / float(n_categories) * 2 * pi for n in range(n_categories)]
    angles += angles[:1]

    # Crear la figura
    fig, ax = plt.subplots(figsize=(12, 10), subplot_kw=dict(polar=True), facecolor='#0E3F5C')

    # Configurar el fondo del gráfico y el rango máximo
    ax.set_facecolor('#0E3F5C')
    ax.set_ylim(0, 10)

    # Dibujar el radar chart para Player 2
    ax.plot(angles, values_player2, color=col2, linewidth=2, linestyle='solid', label=player2)
    ax.fill(angles, values_player2, color=col2, alpha=0.3)

    # Dibujar el radar chart para Player 1
    ax.plot(angles, values_player1, color=col1, linewidth=2, linestyle='solid', label=player1)
    ax.fill(angles, values_player1, color=col1, alpha=0.3)


    # Añadir etiquetas de cada categoría
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories, color='white', size=10, alpha=0.7)

    # Añadir las líneas de los ejes con menor visibilidad
    ax.spines['polar'].set_visible(False)
    ax.yaxis.grid(True, color='white', linestyle='--', linewidth=0.5, alpha=0.5)
    ax.xaxis.grid(True, color='white', linestyle='--', linewidth=0.5, alpha=0.5)

    # Mostrar leyenda en la parte inferior izquierda
    ax.legend(loc='lower left', frameon=False, fontsize=12, labelcolor='white', bbox_to_anchor=(-0.1, 0.01))

    # Añadir el texto "KPIs" en el centro del radar
    ax.text(0, 0, 'KPIs', ha='center', va='center', color='white', fontsize=30, fontweight='bold')

    # Mostrar el gráfico
    plt.tight_layout()
    plt.show()

# Supongamos que df_KPIs_Centrales_Academia es tu DataFrame principal
df = df_KPIs_Centrales_Academia

# Seleccionar los jugadores para comparar
player2 = 'Player.avg'
player1 = 'Alvaro Garcia Blanes'

# Llamar a la función para crear el radar chart
radarchart_KPIs(player1, player2, df)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.patheffects as path_effects

# Variables y configuraciones iniciales
team_name = 'Deportivo Alavés B'
col1 = '#1E90FF'  # Color para player1
col2 = '#D3D3D3'  # Color para player2

# Jugador elegido
player1 = 'Alvaro Garcia Blanes'
player2 = 'Player.avg'

# Función principal para crear el gráfico de rendimiento de los jugadores
def players_Performance_Index(player1, player2, df, ax):
    # Filtrar los datos del DataFrame para los jugadores seleccionados
    df_selected = df[df['player'].isin([player1, player2])]

    # Calcular el índice de rendimiento promedio (Performance_Index)
    df_selected['Performance_Index'] = df_selected.iloc[:, 10:].mean(axis=1)  # KPI columns start from index 10

    # Función para asignar color según el índice de rendimiento
    def get_color(value):
        if value < 3:
            return 'yellow'
        elif 3 <= value < 5:
            return 'lightgreen'
        elif 5 <= value < 8:
            return 'green'
        else:
            return 'darkgreen'

    # Efecto de borde para el texto
    outline_effect = [path_effects.withStroke(linewidth=2, foreground='black')]

    # Configuración de espacio entre cuadros
    spacing = 6

    # Dibujar los cuadros de rendimiento para cada jugador
    for i, (index, row) in enumerate(df_selected.iterrows()):
        performance_value = row['KPI_rendimiento']
        color = get_color(performance_value)

        # Dibujar cuadros redondeados con color
        box = patches.FancyBboxPatch((i * spacing, 0.5), width=3.2, height=1.5, boxstyle="round,pad=0.1", 
                                     edgecolor='black', facecolor=color)
        ax.add_patch(box)

        # Color específico para el jugador
        player_color = col1 if row['player'] == player1 else col2

        # Texto de rendimiento, nombre y valor
        text_performance = ax.text(i * spacing + 1.6, 1.7, "Performance\nIndex", ha="center", va="center", 
                                   fontsize=24, fontweight='bold', color='white')
        text_performance.set_path_effects(outline_effect)

        text_player = ax.text(i * spacing + 1.6, 1.2, row['player'], ha="center", va="center", 
                              fontsize=23, fontweight='bold', color=player_color)
        text_player.set_path_effects(outline_effect)

        text_value = ax.text(i * spacing + 1.6, 0.8, f"{performance_value:.2f}", ha="center", va="center", 
                             fontsize=50, fontweight='bold', color='white')
        text_value.set_path_effects(outline_effect)

        # Añadir la leyenda con los minutos jugados por partido
        play_time_minutes = row['play_time_minutes']
        ax.text(i * spacing + 1.6, 0.3, f"(played minutes per match = {play_time_minutes:.2f})", 
                ha="center", va="center", fontsize=10, color='white')

    # Configuración del gráfico
    ax.set_xlim(-2, spacing * len(df_selected['player']))  # Ajustar límites horizontales
    ax.set_ylim(0, 2.5)  # Ajustar límite vertical
    ax.axis('off')  # Ocultar los ejes

# Crear una figura y asignar un eje
fig, ax = plt.subplots(figsize=(12, 4))
bg_color = '#0E3F5C'  # Color de fondo
fig.patch.set_facecolor(bg_color)
ax.set_facecolor(bg_color)

# Llamar a la función para graficar
players_Performance_Index(player1, player2, df_KPIs_Centrales_Academia, ax)

# Mostrar el gráfico
plt.tight_layout()
plt.show()


In [ ]:

# variables
#Jugador elegido
player1 = 'Alvaro Garcia Blanes'
player2 = 'Player.avg'	
main_position = "CenterBacks"

team_name = "Deportivo Alavés B"
season_name ="24/25"


col1 = '#1E90FF'# player1
col2 = '#D3D3D3'# player2

# OffensiveTatical KPIs  dos equipos
def bars_players_KPIs(ax):
    categories = [ 
        'Indice Rendimiento',
        'Posicionamiento tactico',
        'Capacidad recuperacion',
        'Juego aereo',
        'Eficacia defensiva',
        'Finalizacion',
        'Peligro generado',
        'Habilidad individual',
        'Progresion ofensiva',
        'Construccion ataque',
        'Indice Condicional'
        
         
    ]

    # Valores de los KPIs para cada jugador
    player1_values = [6.32, 3.41, 3.28,  7.96, 3.06, 6.86, 7.96, 7.96, 4.59, 4.59, 3.7]  
    player2_values = [4.87, 4.11,  4.05, 4.07, 3.84, 4.69, 4.59, 5.93, 3.74, 3.74, 4.33]  
    
    # Nombre de los jugadores
    player1 = 'Alvaro Garcia Blanes'
    player2 = 'Player.avg'	


    # Configurar color de fondo
    bg_color = '#0E3F5C'
    fig.patch.set_facecolor(bg_color)
    ax.set_facecolor(bg_color)

    # Configurar las posiciones de las barras
    y = np.arange(len(categories))  # Crear una lista de índices para las categorías

    # Crear las barras horizontales
    ax.barh(y - 0.2, player1_values, height=0.3, label=player1, color=col1)  # Barras para el primer equipo
    ax.barh(y + 0.2, player2_values, height=0.3, label=player2, color=col2)  # Barras para el segundo equipo

    # Etiquetas y leyenda
    ax.set_yticks(y)
    ax.set_yticklabels([f'{cat.replace("_", " ")}' for cat in categories], color='white')
    ax.set_xlabel('KPI Values | "Data from: BePro | Individual Preformance Profile Positions', fontsize=12, color='white',fontweight='bold')
    ax.set_title('CenterBacks Profile KPIs' , color='white',fontsize=25, fontweight='bold')

    # Subtitles
    ax.text(4.5, 10.70, f"({main_position} | {team_name} | {season_name})" , color='white', fontsize=16, ha='center', va='center')
    

    # Añadir valores al final de las barras
    for i, (val1, val2) in enumerate(zip(player1_values, player2_values)):
        ax.text(val1 + 0.1, i - 0.2, f'{val1:.2f}', va='center', color='white', fontsize=20, fontweight='bold')
        ax.text(val2 + 0.1, i + 0.2, f'{val2:.2f}', va='center', color='white', fontsize=20, fontweight='bold')

    # Configurar el color de las líneas y ocultar los ejes
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False)
    ax.set_xticks([])
    ax.set_yticks([])

    # Añadir nombres de KPI sobre el grupo de barras
    for i, category in enumerate(categories):
        ax.text(-0.5, i, category.replace('_', ' '), color='white', fontsize=22, ha='right', va='center')

    # Mostrar leyenda en la parte inferior derecha
    ax.legend(loc='lower left', frameon=False, fontsize=12, facecolor=bg_color, edgecolor=bg_color, labelcolor='white', bbox_to_anchor=(1.0, -0.05))

# Mostrar gráfico
fig,ax=plt.subplots(figsize=(15, 15), facecolor=bg_color)
bars_players_KPIs(ax)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Variables de configuración
team_name = "Deportivo Alavés B"
season_name = "24/25"
main_position = "CenterBacks"

col1 = '#1E90FF'  # Color para player1
col2 = '#D3D3D3'  # Color para player2

# Jugadores seleccionados
player1 = 'Alvaro Garcia Blanes'
player2 = 'Player.avg'

# Función para crear el gráfico de barras horizontal
def bars_players_KPIs(player1, player2, df, ax):
    # Definir las categorías de KPI en el orden deseado
    categories = [
        'KPI_rendimiento',
        'KPI_posicionamiento_tactico',
        'KPI_capacidad_recuperacion',
        'KPI_juego_aereo',
        'KPI_eficacia_defensiva',
        'KPI_finalizacion',
        'KPI_peligro_generado',
        'KPI_habilidad_individual',
        'KPI_progresion',
        'KPI_construccion_ataque',
        'KPI_condicional'
    ]

    # Extraer valores de KPI para los jugadores seleccionados
    player1_values = df.loc[df['player'] == player1, categories].values.flatten()
    player2_values = df.loc[df['player'] == player2, categories].values.flatten()

    # Configuración de color de fondo
    bg_color = '#0E3F5C'
    fig.patch.set_facecolor(bg_color)
    ax.set_facecolor(bg_color)

    # Configuración de posiciones de las barras
    y = np.arange(len(categories))

    # Crear las barras horizontales
    ax.barh(y - 0.2, player1_values, height=0.3, label=player1, color=col1)
    ax.barh(y + 0.2, player2_values, height=0.3, label=player2, color=col2)

    # Etiquetas y leyenda
    ax.set_yticks(y)
    ax.set_yticklabels([cat.replace("KPI_", "").replace("_", " ").title() for cat in categories], fontsize=15, color='white')
    ax.set_xlabel('KPI Values | Data from: BePro | Individual Performance Profile Positions', fontsize=10, color='white', fontweight='bold')
    ax.set_title(f'{main_position} Profile KPIs', color='white', fontsize=25, fontweight='bold')

    # Subtítulos con información del equipo y la temporada
    ax.text(4.5, len(categories) - 0.3, f"({main_position} | {team_name} | {season_name})", color='white', fontsize=12, ha='center', va='center')

    # Añadir valores al final de las barras
    for i, (val1, val2) in enumerate(zip(player1_values, player2_values)):
        ax.text(val1 + 0.1, i - 0.2, f'{val1:.2f}', va='center', color='white', fontsize=12, fontweight='bold')
        ax.text(val2 + 0.1, i + 0.2, f'{val2:.2f}', va='center', color='white', fontsize=12, fontweight='bold')

    # Configurar estilo de gráfico (sin ejes visibles)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False)
    ax.set_xticks([])

    # Mostrar leyenda
    ax.legend(loc='lower left', frameon=False, fontsize=12, facecolor=bg_color, edgecolor=bg_color, labelcolor='white', bbox_to_anchor=(1.0, -0.05))

# Crear la figura y el eje
fig, ax = plt.subplots(figsize=(12, 10), facecolor='#0E3F5C')
bars_players_KPIs(player1, player2, df_KPIs_Centrales_Academia, ax)

# Mostrar el gráfico
plt.tight_layout()
plt.show()


#  Dataframes Rendimiento de los centrales por temporada, comparativa con su categoria 

In [ ]:

##Deportivo ALAVES 

#Definir 
#team ='Deportivo Alavés'
position = 'CB'
season = '23/24'
#league = ''

#Obtener dataframes
df_players_KPIranking= df_KPI_players_posicion[(df_KPI_players_posicion['team_name'].str.startswith('Deportivo Alavés')) & (df_KPI_players_posicion['main_position'] ==position) & (df_KPI_players_posicion['season_name'] == season)]


# Agrupar por jugador único y calcular el promedio de las métricas relacionadas a los partidos
df_players_KPIranking  = df_players_KPIranking.groupby('player').agg({
    'back_number': 'first',  # Número de camiseta
    'player_id': 'first',    # ID del jugador
    'main_position': 'first',  # Posición principal
    'team_name': 'first',     # Nombre del equipo
    'match_id': 'first',      # Cantidad de partidos
    # Promedio de métricas relacionadas al rendimiento (ajusta según los nombres de tus columnas)
    'play_time_minutes' :'mean',
    'KPI_condicional': 'mean',
    'KPI_construccion_ataque': 'mean',
    'KPI_progresion': 'mean',
    'KPI_habilidad_individual': 'mean',
    'KPI_peligro_generado': 'mean',
    'KPI_finalizacion': 'mean',
    'KPI_eficacia_defensiva': 'mean',
    'KPI_juego_aereo': 'mean',
    'KPI_capacidad_recuperacion': 'mean',
    'KPI_posicionamiento_tactico': 'mean',
    'KPI_rendimiento': 'mean'
}).reset_index()

# Mostrar el resultado filtrado
df_players_KPIranking.sort_values(by='KPI_rendimiento', ascending=False).head(20).round(2)








In [ ]:

# Rendimiento de centrales Academia  "Deportivo Alavés"  por 'season_name' 

df_Ranking_CB_Alaves_22_23 = df_filter_CB_Alavés[df_filter_CB_Alavés['team_name'].str.startswith('Deportivo Alavés') & (df_filter_CB_Alavés['season_name'] == '22/23')]
#df_Ranking_CB_Alaves_23_24 = df_filter_CB_Alavés[df_filter_CB_Alavés['team_name'].str.startswith('Deportivo Alavés') & (df_filter_CB_Alavés['season_name'] == '23/24')]



# Agrupar por jugador único y calcular el promedio de las métricas relacionadas a los partidos
df_Ranking_CB_Alaves_22_23  = df_filter_CB_Alavés.groupby('player').agg({
    'back_number': 'first',  # Número de camiseta
    'player_id': 'first',    # ID del jugador
    'main_position': 'first',  # Posición principal
    'team_name': 'first',     # Nombre del equipo
    'match_id': 'first',      # Cantidad de partidos
    # Promedio de métricas relacionadas al rendimiento (ajusta según los nombres de tus columnas)
    'play_time_minutes' :'mean',
    'KPI_comportamiento_actitud': 'mean',
    'KPI_construccion_ataque': 'mean',
    'KPI_progresion': 'mean',
    'KPI_tecnica_individual': 'mean',
    'KPI_peligro_generado': 'mean',
    'KPI_finalizacion': 'mean',
    'KPI_eficacia_defensiva': 'mean',
    'KPI_juego_aereo': 'mean',
    'KPI_capacidad_recuperacion': 'mean',
    'KPI_posicionamiento_tactico': 'mean',
    'KPI_rendimiento': 'mean'
}).reset_index()

# Mostrar el resultado filtrado
df_Ranking_CB_Alaves_22_23.sort_values(by='KPI_rendimiento', ascending=False).head(20).round(2)



In [ ]:
## FunciÓn para visualización de Rendimiento 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from math import pi

# Crear DataFrame
df = pd.DataFrame(df_Ranking_CB_Alaves_22_23)

# Asignar colores por posición
team_name_colors = {
    'Deportivo Alavés Cad. A': 'blue',
    'Deportivo Alavés C': 'pink',
    'Deportivo Alavés B': 'red',
    'Deportivo Alavés Juv. A': 'yellow'
}
df['color'] = df['team_name'].map(team_name_colors)

# Reemplazar NaN en la columna 'color' con un color por defecto, por ejemplo, 'grey'
df['color'] = df['color'].fillna('grey')

# Ordenar los datos de menor a mayor
df = df.sort_values(by='KPI_rendimiento', ascending=True)

# Configurar el fondo del gráfico a gris 
plt.figure(figsize=(12, 8), facecolor='#0E3F5C')

# Graficar en barras horizontales
plt.figure(figsize=(12, 8))
bars = plt.barh(df['player'], df['KPI_rendimiento'], color=df['color'], height=0.5)
plt.xlabel('KPI_rendimiento', color='white')  # Color de la etiqueta
plt.ylabel('Centrales', color='white')  # Color de la etiqueta
plt.title('Performance Index Centrales Academia (Temporada:2023/2024)', color='white')  # Color del título


# Añadir los valores del índice sobre cada barra
for bar in bars:
    xval = bar.get_width()
    plt.text(xval, bar.get_y() + bar.get_height() / 2.0, round(xval, 3), va='center', ha='left', size=10)

# Ajustar el aspecto visual para simular bordes redondeados
for spine in plt.gca().spines.values():
    spine.set_visible(False)

# Añadir leyenda
handles = [plt.Line2D([0], [0], color=color, lw=4, label=team_name) for team_name, color in team_name_colors.items()]
plt.legend(handles=handles, title="Equipos Academia")

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image  # Asegúrate de tener la biblioteca PIL (Pillow) instalada
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import textwrap

# Crear una figura
fig = plt.figure(figsize=(30, 30), facecolor=bg_color)  

# Crear un GridSpec con 4 filas y 2 columnas
gs = gridspec.GridSpec(4, 2, height_ratios=[2, 2, 1.5, 1], width_ratios=[2, 1])



# Asignar subplots a posiciones específicas en la cuadrícula
ax1 = fig.add_subplot(gs[0:2, 0], projection='polar')  # Primera columna, dos primeras filas
ax2 = fig.add_subplot(gs[2, 0])     # Primera columna, tercera fila
ax3 = fig.add_subplot(gs[0, 1])     # Segunda columna, primera fila
ax4 = fig.add_subplot(gs[1, 1])     # Segunda columna, segunda fila
ax5 = fig.add_subplot(gs[2, 1])     # Segunda columna, tercera fila
ax6 = fig.add_subplot(gs[3, 1])     # Segunda columna, cuarta fila


# Llamar a la función teams_TPAC_KPIs en el subplot ax2
players_KPIs(ax2)

# Heading
highlight_text = [{'color': hcol}, {'color': acol}]
fig_text(0.5, 1.0, f"<{teamName}>", color=line_color, fontsize=70, fontweight='bold',
         highlight_textprops=highlight_text, ha='center', va='center', ax=fig)

# Subtitles
fig.text(0.5, 0.96, f"Jor.4, LaLiga 2024-25 | PostMatch Report pag.3", color='white', fontsize=30, ha='center', va='center')
fig.text(0.5, 0.94, f"Data from: Opta | code by: @adnaaan433 | CustomPerformance by: Sevi", color='white', fontsize=22, ha='center', va='center')

# TeamNames
teamName = "Alaves"

# load de images from LOCAL Archive
image_path = f"/Users/sevi/Documents/Escudos/Escudos_LaLiga /{teamName}.png"  # Ruta del escudo 

# Añadir imágenes de los equipos
image = Image.open(aimage_path)
add_image(image, fig, left=0.8, bottom=0.94, width=0.12, height=0.12)

# Añadir visualizaciones
radarchart_KPIs(ax1)  # Visualización para el subplot ax1
players_Performance_Index(ax2)  # Visualización para el subplot ax2
bars_PlayStyle_KPIs(ax3)  # Visualización para el subplot ax3
bars_OffensiveTactical_KPIs(ax4)  # Visualización para el subplot ax4
bars_DefensiveTactical_KPIs(ax5)  # Visualización para el subplot ax5
bars_EfficacyTeams_KPIs(ax6)  # Visualización para el subplot ax6

# Etiquetas para identificar los subplots
ax1.set_title('', color='white', fontsize=28, fontweight='bold')
ax2.set_title("\n"
        "TPAC Index measures the competitive performance of a team,\n" 
        "based on the behavior and effectiveness\n"
        "shown in relation to the tactical aspects\n"
        "with and without the ball, which occur in a competition", color='white', fontsize=20, fontweight='bold')
ax3.set_title('Play Style KPIs', color='white', fontsize=28, fontweight='bold')
ax4.set_title('Offensive Tactical KPIs', color='white', fontsize=28, fontweight='bold')
ax5.set_title('Defensive Tactical KPIs', color='white', fontsize=28, fontweight='bold')
ax6.set_title('Efficacy Teams KPIs', color='white', fontsize=28, fontweight='bold')


# Ajustar los márgenes y espacios entre subplots
plt.tight_layout()
fig.subplots_adjust(left=0.05, right=0.95, top=0.90, bottom=0.05, hspace=0.3, wspace=0.)


#Añadir texto de analisis 
fig.text(0.16, 0.19, f"GENERAL TACTICAL ANALYSIS:", color='white', fontsize=30, fontweight='bold', ha='center', va='center')
fig.text(0.01, 0.07, f"En un primer analisis podemos observar como el Rendimiento de ambos equipos es parecido onbservamos\n"
"una clara diferencia de estilos, el Celta domina la posesión, y su estilo de ataque es combinativo\n"
"construyendo desde atrás como determina claramennte los valores de BuildUp Attacks y Build Up GK , \n"
"influence, mientras que Osasuna se muestra como un equipo que progresa en ataque con menor numero de\n"
"pases y un juego de estiolo más directo.\n"
"Sin balón, sus estilos son presionantes con bloque defensivos medios-altos,sin embargo,durante el partido\n"
"Osasuna fue capaz de recuperar mas veces y más arriba,con una presión más eficaz.\n"
"En zona de finalización celta generó mas y mas claras ocasiones llevando mas peligro,pero sin embargo el\n" 
"resultado se decantó del lado navarro, al mostar más eficacia, la linea de Rendimiento del Ceta es adecuada \n"
"pero con areas de mejora en finalización." , color='white', fontsize=20)


# Guardar la figura
# fig.savefig(f"/Users/sevi/Documents/PostMaches/Partidos/FData_PostMatches_reports/Match_Report_3.png", bbox_inches='tight')

# Mostrar la figura
plt.show()


In [ ]:
from IPython.display import Image

# Mostrar la imagen desde la ruta local
Image(filename='/Users/sevi/Documents/Performance Report.png')


In [ ]:
#Promedio de centrales con mejor rendimiento 
unique_CenterBacks= df_filter_match_Alavés.drop_duplicates(subset='player')
unique_CenterBacks.head(20)

In [ ]:
# Filtrar el DataFrame para obtener filas únicas basadas en 'team_id'
unique_season = df_KPI_centrales.drop_duplicates(subset='season_name')
unique_season.head(20)